## Unit testing the EWR tool

In [8]:
import unittest
import pandas as pd
from pandas._testing import assert_frame_equal
from datetime import datetime, date
import copy
from pathlib import Path
import os

import requests
import io

import re
import numpy as np

from mdba_gauge_getter import gauge_getter
from py_ewr import dashboard, data_inputs,scenario_handling, observed_handling, evaluate_EWRs, summarise_results

BASE_PATH = Path(os.path.abspath('')).resolve()

### Testing function in summarise_results.py module

In [2]:
class test_summarise_results(unittest.TestCase):
    def test_sum_events(self):
        '''
        Series input to summarise_results function will only be made up of combination of 1's or 0's. 
        1. Test to see if function is counting occurences of 1's
        '''
        # Test 1
        input_series = pd.Series(index=[1895, 1896, 1897, 1898, 1899], data=[0,1,1,1,0])
        s = summarise_results.sum_events(input_series)
        expected_s = 3
        self.assertEqual(s, expected_s)
    
    
    def test_get_frequency(self):
        '''
        Series input to get_frequency function will only be made up of combination of 1's or 0's.
        1. Test to see if function is returning frequency of 1 occurence out of all years. 
        2. Test to see if function handles no occurence of 1's
        '''
        # Test 1
        input_series = pd.Series(index=[1895, 1896, 1897, 1898, 1899], data=[0,1,1,1,0])
        f = summarise_results.get_frequency(input_series)
        expected_f = 60
        self.assertEqual(f, expected_f)
        # ----------------------------
        # Test 2
        input_series = pd.Series(index=[1895, 1896, 1897, 1898, 1899], data=[0,0,0,0,0])
        f = summarise_results.get_frequency(input_series)
        expected_f = 0
        self.assertEqual(f, expected_f)

    def test_get_average(self):
        '''
        Input will be a series of years and either integers or floats
        1. Test average
        2. Test average if all numbers in series are 0
        '''
        # Test 1
        input_series = pd.Series(index=[1895, 1896, 1897, 1898, 1899], data=[100,100,50,50,0])
        f = summarise_results.get_average(input_series)
        expected_f = 60
        self.assertEqual(f, expected_f)
        # ----------------------------
        # Test 2
        input_series = pd.Series(index=[1895, 1896, 1897, 1898, 1899], data=[0,0,0,0,0])
        f = summarise_results.get_average(input_series)
        expected_f = 0
        self.assertEqual(f, expected_f)
        
    def test_event_length(self):
        '''
        Input is a tuple containing dictionaries of events. Typical tuples contain one dictionary, however, for the EWRs that require flows to be met across multiple sites, there will be multiple event dictionaries within the tuple
        1. Test getting the average event length over a multi site EWR
        2. Test getting the average event length over a single site EWR
        '''
        # Test 1
        events1 = {2012: [[50,50,50,100,100,100],[5,5,7.5,10,10]], 2013: [[50,50,100,100],[5,5,7.5,10,10]], 2014: [[50,50,50,100,100,100],[5,5,7.5,10,10]]}
        events2 = {2012: [[50,50,50,100,100,100]], 2013: [[50,50,100,100],[5,5,7.5,10,10]], 2014: []}
        tupled = tuple([events1, events2])
        EWR = {}
        EWR['VL'] = tupled
        PU = {}
        PU['planning unit 1'] = EWR
        gauge = {}
        gauge['425001'] = PU
        scenario = {}
        scenario['scenario 1'] = gauge
        f = summarise_results.get_event_length(scenario['scenario 1']['425001']['planning unit 1']['VL'])
        
        expected_f = 5.111111111111111
        self.assertEqual(expected_f, f)
        #------------------------------
        # Test 2
        events1 = {2012: [[50,50,50,100,100,100],[5,5,7.5,10,10]], 2013: [[50,50,100,100],[5,5,7.5,10,10]], 2014: [[50,50,50,100,100,100],[5,5,7.5,10,10]]}
        tupled = tuple([events1])
        EWR = {}
        EWR['VL'] = tupled
        PU = {}
        PU['planning unit 1'] = EWR
        gauge = {}
        gauge['425001'] = PU
        scenario = {}
        scenario['scenario 1'] = gauge

        f = summarise_results.get_event_length(scenario['scenario 1']['425001']['planning unit 1']['VL'])
        
        expected_f = 5.166666666666667
        self.assertEqual(expected_f, f)        
        
    def test_get_threshold_days(self):
        '''
        Input is a tuple containing dictionaries of events. Typical tuples contain one dictionary, however, for the EWRs that require flows to be met across multiple sites, there will be multiple event dictionaries within the tuple
        1. Test getting the total event length of multi site EWR
        2. Test getting the total event length over a single site EWR
        '''
        # Test 1
        events1 = {2012: [[50,50,50,100,100,100],[5,5,7.5,10,10]], 2013: [[50,50,100,100],[5,5,7.5,10,10]], 2014: [[50,50,50,100,100,100],[5,5,7.5,10,10]]}
        events2 = {2012: [[50,50,50,100,100,100]], 2013: [[50,50,100,100],[5,5,7.5,10,10]], 2014: []}
        tupled = tuple([events1, events2])
        EWR = {}
        EWR['VL'] = tupled
        PU = {}
        PU['planning unit 1'] = EWR
        gauge = {}
        gauge['425001'] = PU
        scenario = {}
        scenario['scenario 1'] = gauge
        f = summarise_results.get_event_length(scenario['scenario 1']['425001']['planning unit 1']['VL'])
        
        expected_f = 5.111111111111111
        self.assertEqual(expected_f, f)
        #------------------------------
        # Test 2
        events1 = {2012: [[50,50,50,100,100,100],[5,5,7.5,10,10]], 2013: [[50,50,100,100],[5,5,7.5,10,10]], 2014: [[50,50,50,100,100,100],[5,5,7.5,10,10]]}
        tupled = tuple([events1])
        EWR = {}
        EWR['VL'] = tupled
        PU = {}
        PU['planning unit 1'] = EWR
        gauge = {}
        gauge['425001'] = PU
        scenario = {}
        scenario['scenario 1'] = gauge

        f = summarise_results.get_event_length(scenario['scenario 1']['425001']['planning unit 1']['VL'])
        
        expected_f = 5.166666666666667
        self.assertEqual(expected_f, f)  
        
        
    def test_count_exceedence(self):
        '''
        Input is a series with water years along the index, and a list of lists, containing integers for the data.
        1. Test number of exceedences when there is a max interevent to be exceeded
        2. Test the function returns 'N/A' when there is no max interevent to be checked event for this EWR
        '''
        # Test 1
        index = [1895, 1896, 1897, 1898, 1899, 1900]
        data = [[15, 30], [], [], [450, 2], [10,12], [200,10]]
        series = pd.Series(index=index, data=data)

        EWR_info = {'max_inter-event': 2}

        result = summarise_results.count_exceedence(series, EWR_info)
        expected_result = 8 # There are 8 exceedences in the above list
        self.assertEqual(result, expected_result)
        # ---------------------------------------
        # Test 2
        index = [1895, 1896, 1897, 1898, 1899, 1900]
        data = [[15, 30], [], [], [450, 2], [10,12], [200,10]]
        series = pd.Series(index=index, data=data)

        EWR_info = {'max_inter-event': None}

        result = summarise_results.count_exceedence(series, EWR_info)
        expected_result = 'N/A' # There is no max interevent period defined so cannot return a result
        self.assertEqual(result, expected_result)
        
    def test_initialise_summary_df_columns(self):
        '''
        Input is a dictionary of scenario EWR results for each gauge and planning unit.
        1. Test proper initialisation of multi index columns
        '''
        # Test 1
        # Input data and send to function
        columns = ['Event years','Frequency','Target frequency','Achievement count', 'Achievements per year', 'Event count','Events per year',
                    'Event length','Threshold days','Inter-event exceedence count', 'Max inter event period (years)', 'No data days',
                    'Total days']
        df = pd.DataFrame()
        df1 = pd.DataFrame()
        input_dictionary = {'Scenario 1': {'Gauge 1': {'Planning unit 1': df}}, 'Scenario 2': {'Gauge 1': {'Planning unit 1': df1}}}

        multi_index = summarise_results.initialise_summary_df_columns(input_dictionary)
        # Define expected output given the above input
        tuples = [('Scenario 1', 'Event years'),
                  ('Scenario 1', 'Frequency'),
                  ('Scenario 1', 'Target frequency'),
                  ('Scenario 1', 'Achievement count'),
                  ('Scenario 1', 'Achievements per year'),
                  ('Scenario 1', 'Event count'),
                  ('Scenario 1', 'Events per year'),
                  ('Scenario 1', 'Event length'),
                  ('Scenario 1', 'Threshold days'),
                  ('Scenario 1', 'Inter-event exceedence count'),
                  ('Scenario 1', 'Max inter event period (years)'),
                  ('Scenario 1', 'No data days'),
                  ('Scenario 1', 'Total days'),
                  ('Scenario 2', 'Event years'),
                  ('Scenario 2', 'Frequency'),
                  ('Scenario 2', 'Target frequency'),
                  ('Scenario 2', 'Achievement count'),
                  ('Scenario 2', 'Achievements per year'),
                  ('Scenario 2', 'Event count'),
                  ('Scenario 2', 'Events per year'),
                  ('Scenario 2', 'Event length'),
                  ('Scenario 2', 'Threshold days'),
                  ('Scenario 2', 'Inter-event exceedence count'),
                  ('Scenario 2', 'Max inter event period (years)'),
                  ('Scenario 2', 'No data days'),
                  ('Scenario 2', 'Total days')]
        expected_multi_index = pd.MultiIndex.from_tuples(tuples, names=['scenario', 'type'])
        
        # test result:
        for i, tup in enumerate(expected_multi_index):
            for index, val in enumerate(tup):
                self.assertEqual(multi_index[i][index], val)
                
    def test_initialise_summary_df_rows(self):
        '''
        Input is a dictionary of scenario EWR results for each gauge and planning unit.
        1. Test for expected initialisation of three layered multi index
        '''
        # Test 1
        # Setting up input data and sending to function:
        columns = ['CF1_eventYears','CF1_numAchieved','CF1_numEvents','CF1_eventLength','CF1_totalEventDays','CF1_daysBetweenEvents','CF1_missingDays','CF1_totalPossibleDays','VF1_eventYears',
         'VF1_numAchieved','OB-L1_S_missingDays','OB-L1_S_totalPossibleDays','OB-L1_P_eventYears','OB-L1_P_numAchieved','OB-L1_P_numEvents','OB-L1_P_eventLength']
        df = pd.DataFrame(columns=columns)
        input_dictionary = {'Scenario 1': {'Gauge 1': {'Planning unit 1': df}}, 'Scenario 2': {'Gauge 1': {'Planning unit 1': df}}}

        multi_index = summarise_results.initialise_summary_df_rows(input_dictionary)

        # Setting up expected output:
        tuples = [('Gauge 1', 'Planning unit 1', 'CF1'), ('Gauge 1', 'Planning unit 1', 'VF1'), ('Gauge 1', 'Planning unit 1', 'OB-L1_S'), ('Gauge 1', 'Planning unit 1', 'OB-L1_P')]
        expected_multi_index = pd.MultiIndex.from_tuples(tuples, names=['gauge', 'planning unit', 'EWR'])
        
        # Test result:
        for i, tup in enumerate(expected_multi_index):
            for index, val in enumerate(tup):
                self.assertEqual(multi_index[i][index], val)        
        
    def test_allocate(self):
        '''
        Input is a dataframe to save information to, the information to be saved, and the coordinates of the dataframe
        1. Test information is being saved in the expected location in the dataframe
        '''
        # Test 1
        # Define input data and send to function:
        tuples = [('Scenario 1', 'Event years'),
                  ('Scenario 1', 'Frequency'),
                  ('Scenario 2', 'No data days'),
                  ('Scenario 2', 'Total days')]
        multi_column = pd.MultiIndex.from_tuples(tuples, names=['scenario', 'type'])        
        tuples = [('Gauge 1', 'Planning unit 1', 'CF1'), 
                  ('Gauge 1', 'Planning unit 1', 'VF1'), 
                  ('Gauge 1', 'Planning unit 1', 'OB-L1_S'), 
                  ('Gauge 1', 'Planning unit 1', 'OB-L1_P')]
        multi_index = pd.MultiIndex.from_tuples(tuples, names=['gauge', 'planning unit', 'EWR'])
        df = pd.DataFrame(index = multi_index, columns=multi_column)
        add_this = 12
        idx = pd.IndexSlice
        site = 'Gauge 1'
        PU = 'Planning unit 1'
        EWR = 'OB-L1_S'
        scenario = 'Scenario 2'
        category = 'No data days'
        result_df = summarise_results.allocate(df, add_this, idx, site, PU, EWR, scenario, category)
        # Set expected dataframe
        expected_df = pd.DataFrame(index = multi_index, columns=multi_column)
        expected_df.loc[idx[[site], [PU], [EWR]], idx[scenario, category]] = 12
        # Test result:
        assert_frame_equal(result_df, expected_df)
        
    def test_summarise(self):
        '''
        Inputs are a dictionary containing annualised results for each Scenario>Gauge>Planning Unit>EWR and 
        a dictionary containing individual event information for each Scenario>Gauge>Planning Unit>EWR
        1. Test each part of the function are working correctly and producing an overall expected output
        '''
        
        # Test 1
        # setting up input data and sending to function
        df = pd.read_csv('unit_testing_files/input_for_summarise_data.csv', index_col  =0)
        for col in df:
            if 'daysBetweenEvents' in col:
                for i, val in enumerate(df[col]):
                    new = df[col].iloc[i]
                    if new == '[]':
                        new_list = []
                    else:
                        new = re.sub('\[', '', new)
                        new = re.sub('\]', '', new)
                        new = new.split(',')
                        new_list = []
                        for days in new:
                            new_days = days.strip()
                            new_days = int(new_days)
                            new_list.append(new_days)

                    df[col].iloc[i] = new_list
        input_dict = {'Scenario 1': {'410007': {'Upper Yanco Creek': df}}}
        input_events = {'Scenario 1': {'410007': {'Upper Yanco Creek': {'VF1': tuple([{1895: [[400]*100, [400]*50],
                                                                               1896: [], 
                                                                               1897: [[500]*50, [400]*30], 
                                                                               1898: [], 
                                                                               1899: [], 
                                                                               1900: [[500]*345]}])}}}}
        result = summarise_results.summarise(input_dict, input_events)
        # Set up expected outputs and test
        tuples = [('Scenario 1', 'Event years'),
                  ('Scenario 1', 'Frequency'),
                  ('Scenario 1', 'Target frequency'),
                  ('Scenario 1', 'Achievement count'),
                  ('Scenario 1', 'Achievements per year'),
                  ('Scenario 1', 'Event count'),
                  ('Scenario 1', 'Events per year'),
                  ('Scenario 1', 'Event length'),
                  ('Scenario 1', 'Threshold days'),
                  ('Scenario 1', 'Inter-event exceedence count'),
                  ('Scenario 1', 'Max inter event period (years)'),
                  ('Scenario 1', 'No data days'),
                  ('Scenario 1', 'Total days')]
        multi_column = pd.MultiIndex.from_tuples(tuples, names=['scenario', 'type'])        
        tuples = [('410007', 'Upper Yanco Creek', 'VF1')]
        multi_index = pd.MultiIndex.from_tuples(tuples, names=['gauge', 'planning unit', 'EWR'])
        expected_df = pd.DataFrame(index = multi_index, columns=multi_column)
        expected_df.iloc[0]=[3, 50, str(100), 3, 0.5, 3, 0.5, 115.0, 575, 0, 0.010958904, 0, 2191]        
        
        assert_frame_equal(result, expected_df)
        
Test = test_summarise_results()
suite = unittest.TestSuite()

suite.addTest(test_summarise_results('test_sum_events'))
suite.addTest(test_summarise_results('test_get_frequency'))
suite.addTest(test_summarise_results('test_get_average'))
suite.addTest(test_summarise_results('test_event_length'))
suite.addTest(test_summarise_results('test_count_exceedence'))
suite.addTest(test_summarise_results('test_initialise_summary_df_columns'))
suite.addTest(test_summarise_results('test_initialise_summary_df_rows'))
suite.addTest(test_summarise_results('test_allocate'))
suite.addTest(test_summarise_results('test_summarise'))

unittest.TextTestRunner().run(suite)

......../tmp/ipykernel_12590/2379932420.py:289: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col].iloc[i] = new_list
.
----------------------------------------------------------------------
Ran 9 tests in 0.895s

OK


<unittest.runner.TextTestResult run=9 errors=0 failures=0>

### Testing functions in data_inputs.py module

In [12]:
class test_data_inputs(unittest.TestCase):
    def test_convert_max_interevent(self):
        '''
        1. Test max interevent is converted from years to days
        '''
        # Test 1
        unique_water_years = [2012, 2013, 2014, 2015]
        water_years = [2012]*365+[2013]*365+[2014]*365+[2015]*365
        EWR_info = {'max_inter-event': 1}
        new_max_interevent = data_inputs.convert_max_interevent(unique_water_years, water_years, EWR_info)
        self.assertEqual(new_max_interevent, 365)
        
        
    def test_gauge_to_catchment(self):
        '''
        1. Test gauge in Gwydir returns Gwydir
        2. Test gauge in Namoi returns Namoi
        '''
        # Test 1
        expected_catchment = 'Gwydir'
        input_gauge = '418026'
        catchment = data_inputs.gauge_to_catchment(input_gauge)
        self.assertEqual(catchment, expected_catchment)
        # Test 2
        expected_catchment = 'Namoi'
        input_gauge = '419049'
        catchment = data_inputs.gauge_to_catchment(input_gauge)
        self.assertEqual(catchment, expected_catchment)
        
        
    def test_wy_to_climate(self):
        '''
        1. Test an array of climate categorisations are returned for each day of the input data
        '''
        # Setting up input data 
        data = {'418026': list(range(0,730,1)), '425012': list(range(0,1460,2))}
        index = pd.date_range(start='1/1/2017', end='31/12/2018')
        df = pd.DataFrame(data = data, index = index)
        water_years = np.array([2017]*365+[2018]*365)
        climate_daily = data_inputs.wy_to_climate(water_years, 'Gwydir', 'Standard - 1911 to 2018 climate categorisation')
        # Setting up expected output data and the test
        expected_climate_daily = np.array(['Dry']*365 + ['Very Dry']*365)
        self.assertTrue(np.array_equal(climate_daily, expected_climate_daily))
        
        
    def test_get_multi_gauges(self):
        '''
        1. Test for returning planning units and gauges where there are multi gauge EWR requirements
        2. Test for returning the unique gauge to gauge dictionaries where there are multi gauge EWR requirements
        '''
        # Test 1
        expected_multi_gauges = {'PU_0000130': {'421090': '421088', '421088': '421090'},
                                 'PU_0000131': {'421090': '421088', '421088': '421090'},
                                 'PU_0000132': {'421090': '421088', '421088': '421090'},
                                 'PU_0000133': {'421090': '421088', '421088': '421090'}}
        multi_gauges = data_inputs.get_multi_gauges('all')
        self.assertDictEqual(expected_multi_gauges, multi_gauges)
        #--------------------------------------------------------
        # Test 2
        expected_multi_gauges = {'421090': '421088', '421088': '421090'}
        multi_gauges = data_inputs.get_multi_gauges('gauges')
        self.assertDictEqual(expected_multi_gauges, multi_gauges)
        
        
    def test_get_simultaneous_gauges(self):
        '''
        1. Test for returning planning units and gauges where there are simultaneous gauge EWR requirements
        2. Test for returning the unique gauge to gauge dictionaries where there are simultaneous gauge EWR requirements
        '''
        # Testing the return all request
        expected_sim_gauges = {'PU_0000131': {'421090': '421022', '421022': '421090'},
                               'PU_0000132': {'421090': '421022', '421022': '421090'},
                               'PU_0000133': {'421090': '421022', '421022': '421090'}}
        sim_gauges = data_inputs.get_simultaneous_gauges('all')
        self.assertDictEqual(expected_sim_gauges, sim_gauges)
        #----------------------------------------------------
        # Testing the return just gauges request
        expected_sim_gauges = {'421090': '421022', '421022': '421090'}
        sim_gauges = data_inputs.get_simultaneous_gauges('gauges')
        self.assertDictEqual(expected_sim_gauges, sim_gauges)
        
        
    def test_get_climate_cats(self):
        '''
        - 1. Loading the correct climate cat csv
        '''
        # Test 1
        climate_file = 'Standard - 1911 to 2018 climate categorisation'
        result = data_inputs.get_climate_cats(climate_file)
        expected_result = pd.read_csv( BASE_PATH / 'py_ewr'/'climate_data/climate_cats.csv', index_col = 0)
        assert_frame_equal(result, expected_result)
        
    
    def test_get_EWR_table(self):
        '''
        - 1. Test for equal entries (no lost EWRs)
        - 2. Test to ensure no bad EWRs make it through using a subset of EWRs
        '''
        # Test 1
        proxies={} # Populate with your proxy settings
        my_url = 'https://az3mdbastg001.blob.core.windows.net/mdba-public-data/NSWEWR_LIVE.csv'
        s = requests.get(my_url, proxies=proxies).text
        df = pd.read_csv(io.StringIO(s),
                         usecols=['PlanningUnitID', 'PlanningUnitName',  'CompliancePoint/Node', 'gauge', 'code', 'start month',
                                  'end month', 'frequency', 'events per year', 'duration', 'min event', 'flow threshold min', 'flow threshold max',
                                  'max inter-event', 'within event gap tolerance', 'weirpool gauge', 'flow level volume', 'level threshold min',
                                  'level threshold max', 'volume threshold', 'drawdown rate'],
                         dtype='str'
                        )
        
        # Get the cleaned dataset:
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        
        total_len = len(EWR_table)+len(bad_EWRs)
        self.assertEqual(len(df), total_len)
        #-----------------------------------
        # Test 2
        # Send the dummy data through the function
        dummy_EWR_table = 'https://az3mdbastg001.blob.core.windows.net/mdba-public-data/test_EWR_table_input.csv'
        good, bad = data_inputs.get_EWR_table(dummy_EWR_table)
        
        # Load in expected output
        expected_good = pd.read_csv('unit_testing_files/test_EWR_table_good_output.csv', dtype = 'str', encoding='cp1252')
        expected_good = expected_good.fillna('')
        expected_good['flow threshold max'].replace({'':'1000000'}, inplace = True)
        expected_good['level threshold max'].replace({'':'1000000'}, inplace = True)
        expected_bad = pd.read_csv('unit_testing_files/test_EWR_table_bad_output.csv', dtype = 'str', encoding='cp1252')
        expected_bad = expected_bad.fillna('')
        
        assert_frame_equal(expected_good.reset_index(drop=True), good.reset_index(drop=True))
        assert_frame_equal(expected_bad.reset_index(drop=True), bad.reset_index(drop=True))
        
        
    def test_map_gauge_to_catchment(self):
        '''
        1. Run test data (stored on MDBA public data repository) through to see if gauges are mapping correctly
        '''
        
        dummy_EWR_table = 'https://az3mdbastg001.blob.core.windows.net/mdba-public-data/test_EWR_table_input.csv'
        
        result = data_inputs.map_gauge_to_catchment(dummy_EWR_table)
        expected_result = {'Namoi': {}, 
                           'Gwydir': {'418013': 'Gwydir at Gravesend '}, 
                           'Macquarie-Castlereagh': {'421004': 'Warren Weir ', '421090': 'Flows are to be met at both the gauge on the Macquarie River downstream Marebone and at Oxley Station ', 
                                                     '421022': 'Flows are to be met at both the gauge on the Macquarie River downstream Marebone and at Oxley Station '}, 
                           'Lachlan': {}, 
                           'Lower Darling': {'425020': 'Lake Wetherell & Tandure '}, 
                           'Barwon-Darling': {}, 
                           'Murray': {}, 
                           'Murrumbidgee': {}, 
                           'Border Rivers': {}, 
                           'Moonie': {}, 
                           'Condamine-Balonne': {}, 
                           'Warrego': {}, 
                           'Paroo': {},
                          }
        
        self.assertDictEqual(result, expected_result)      
        
Test = test_data_inputs()
suite = unittest.TestSuite()

suite.addTest(test_data_inputs('test_convert_max_interevent'))
suite.addTest(test_data_inputs('test_gauge_to_catchment'))
suite.addTest(test_data_inputs('test_wy_to_climate'))
suite.addTest(test_data_inputs('test_get_multi_gauges'))
suite.addTest(test_data_inputs('test_get_simultaneous_gauges'))
suite.addTest(test_data_inputs('test_get_climate_cats'))
suite.addTest(test_data_inputs('test_get_EWR_table'))
suite.addTest(test_data_inputs('test_get_EWR_table'))
suite.addTest(test_data_inputs('test_map_gauge_to_catchment'))


unittest.TextTestRunner().run(suite)

../home/pedro/.pyenv/versions/3.8.13/lib/python3.8/unittest/case.py:633: UserWarning: Parsing '31/12/2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  method()
.......
----------------------------------------------------------------------
Ran 9 tests in 2.821s

OK


<unittest.runner.TextTestResult run=9 errors=0 failures=0>

### Testing the observed_handling.py module

In [13]:
class test_observed_handling(unittest.TestCase):
    def test_observed_cleaner(self):
        '''
        1. Run sample data through and compare the expected output sample data
        '''
        # Test 1
        # Load sample input data and pass to function
        input_df = pd.read_csv( 'unit_testing_files/observed_flows_test_input.csv')
        dates = {'start_date': date(2014, 1, 1), 'end_date': date(2020, 1, 1)}
        result = observed_handling.observed_cleaner(input_df, dates)
        # Load sample output data and test
        output_df = 'unit_testing_files/observed_flows_test_output.csv'
        expected_result = pd.read_csv(output_df, index_col = 'Date')
        expected_result.index = pd.to_datetime(expected_result.index, format='%Y-%m-%d')
        expected_result.columns = ['419039']
        assert_frame_equal(result, expected_result)
    
    def test_one_gauge_per_column(self):
        '''
        1. Run sample data through and compare to the expected output sample data
        '''
        # Test 1
        # load and format expected output data
        output_df = 'unit_testing_files/observed_flows_test_output.csv'
        expected_result = pd.read_csv(output_df, index_col = 'Date')
        expected_result.index = pd.to_datetime(expected_result.index, format='%Y-%m-%d')
        expected_result.columns = ['419039']
        # Load input data and pass to function
        input_dataframe = pd.read_csv('unit_testing_files/observed_flows_test_input.csv')
        gauge_iter = 419039
        input_dataframe['Date'] = expected_result.index
        result = observed_handling.one_gauge_per_column(input_dataframe, gauge_iter)
        # assert equal test
        assert_frame_equal(result, expected_result)
        
        
    def test_remove_data_with_bad_QC(self):
        '''
        1. Run sample data through and compare to the expected output sample data to ensure bad data is removed
        '''        
        # Test 1
        # Load sample input data:
        input_dataframe = pd.read_csv('unit_testing_files/observed_flows_test_input_QC.csv', index_col = 'Date')
        gauge_iter = 419039
        qc_codes = data_inputs.get_bad_QA_codes()
        # Load output sample data
        expected_df = pd.read_csv('unit_testing_files/observed_flows_test_output_QC.csv', index_col = 'Date')

        # Throw to the function
        df = observed_handling.remove_data_with_bad_QC(input_dataframe, qc_codes)
        
        # Test for equality
        assert_frame_equal(df, expected_df)
    
    
    def test_categorise_gauges(self):
        '''
        1. gauges in all categories
        2. gauges outside cats
        '''
        
        level = ['425020', '425022', '425023', '414209', '4260501', '4260508','4260506']
        flow = ['414203', '425010', '4260507', '4260505',  '421090', '421088', '421088', '421090', '421090', '421088', '421088', '421090', '421090', '421088', '421088', '421090', '421090', '421088', '421088', '421090', '409023', '409003']
        all_gauges = level + flow
        
        f, l = observed_handling.categorise_gauges(all_gauges)

        expected_level = copy.deepcopy(level)
        expected_flow = copy.deepcopy(flow)
        expected_flow = expected_flow + ['421022'] # Add in this one as it will be getting picked up for being associated with a simultaneious gauge
        self.assertEqual(set(f), set(expected_flow))
        self.assertEqual(set(l), set(expected_level))
        
        
    def test_observed_handler(self):
        '''
        1. Test each part of the function are working correctly and producing an overall expected output
        '''
        
        # Set up input parameters and pass to test function
        gauges = ['419039']
        dates = {'start_date': date(2020, 7, 1), 'end_date': date(2021, 6, 30)}
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'

        detailed, summary = observed_handling.observed_handler(gauges, dates, allowance, climate)
        
        # Load and format expected results
        expected_detailed_results = pd.read_csv('unit_testing_files/detailed_results_observed.csv', index_col = 0)
        expected_detailed_results.index = expected_detailed_results.index.astype('object')
        cols = expected_detailed_results.columns[expected_detailed_results.columns.str.contains('eventLength')]
        expected_detailed_results[cols] = expected_detailed_results[cols].astype('float64')
        for col in expected_detailed_results:
            if 'daysBetweenEvents' in col:
                for i, val in enumerate(expected_detailed_results[col]):
                    new = expected_detailed_results[col].iloc[i]
                    if new == '[]':
                        new_list = []
                    else:
                        new = re.sub('\[', '', new)
                        new = re.sub('\]', '', new)
                        new = new.split(',')
                        new_list = []
                        for days in new:
                            new_days = days.strip()
                            new_days = int(new_days)
                            new_list.append(new_days)

                    expected_detailed_results[col].iloc[i] = new_list
        
        assert_frame_equal(detailed['observed']['419039']['Boggabri to Wee Waa'], expected_detailed_results)


        
Test = test_observed_handling()
suite = unittest.TestSuite()

suite.addTest(test_observed_handling('test_observed_cleaner'))
suite.addTest(test_observed_handling('test_one_gauge_per_column'))
suite.addTest(test_observed_handling('test_remove_data_with_bad_QC'))
suite.addTest(test_observed_handling('test_categorise_gauges'))
suite.addTest(test_observed_handling('test_observed_handler'))


unittest.TextTestRunner().run(suite)

....INFO:mdba_gauge_getter.gauge_get:NSW - Request 1 of 1


https://realtimedata.waternsw.com.au/cgi/webservice.pl?{"params":{"site_list":"419039","start_time":"20200701000000","varfrom":"100.00","interval":"day","varto":"141.00","datasource":"CP","end_time":"20210630000000","data_type":"mean","multiplier":"1"},"function":"get_ts_traces","version":"2"}


/tmp/ipykernel_12590/2967290611.py:109: SettingWithCopyWarning:                                                                                                                                      
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  expected_detailed_results[col].iloc[i] = new_list
.
----------------------------------------------------------------------
Ran 5 tests in 6.522s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

### Testing the scenario_handling.py module

In [14]:
class test_scenario_handling(unittest.TestCase):
    def test_match_MDBA_nodes(self):
        '''
        1. Ensure dataframe with flows and levels is split into two dataframes (one flow and one level dataframe)
        '''
        # Set up input data and pass to test function:
        model_metadata = data_inputs.get_MDBA_codes()
        data_df = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                   'EUSTDS-1-8': [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10,
                   'EUSTUS-35-8': [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10
                  }
        df = pd.DataFrame(data = data_df)
        df = df.set_index('Date')
        
        df_F, df_L = scenario_handling.match_MDBA_nodes(df, model_metadata)
        
        # Set up expected outputs and test:
        data_expected_df_L = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                              '414209': [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10
                             }
        expected_df_L = pd.DataFrame(data_expected_df_L)
        expected_df_L = expected_df_L.set_index('Date') 
        data_expected_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                              '414203': [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10
                             }
        expected_df_F = pd.DataFrame(data_expected_df_F)
        expected_df_F = expected_df_F.set_index('Date')
        
        assert_frame_equal(df_F, expected_df_F)
        assert_frame_equal(df_L, expected_df_L)
    
    def test_match_NSW_nodes(self):
        '''
        1. Check NSW model nodes are mapped correctly to their gauges
        '''
        # Set up input data and pass to test function:
        model_metadata = data_inputs.get_NSW_codes()
        data_df = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                   'Gauge: YCB_410134_BillabongCreek@Darlot: Downstream Flow': [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10,
                   'Gauge: YCB_410016 Billabong Creek @ Jerilderie: Downstream Flow': [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10
                  }
        df = pd.DataFrame(data = data_df)
        df = df.set_index('Date')
        
        df_F, df_L = scenario_handling.match_NSW_nodes(df, model_metadata)
        
        # Set up expected outputs and test:
        data_expected_df_L = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))}
        expected_df_L = pd.DataFrame(data_expected_df_L)
        expected_df_L = expected_df_L.set_index('Date')
        
        data_expected_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                              '410134': [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10,
                              '410016': [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10
                             }
        expected_df_F = pd.DataFrame(data_expected_df_F)
        expected_df_F = expected_df_F.set_index('Date')
        
        assert_frame_equal(df_F, expected_df_F)
        assert_frame_equal(df_L, expected_df_L)
    
    def test_extract_gauge_from_string(self):
        '''
        1. Check gauge string is pulled from various strings likely encountered during standard program run
        '''
        # Test 1
        input_string = '409025'
        return_gauge = scenario_handling.extract_gauge_from_string(input_string)
        expected_gauge = '409025'
        self.assertEqual(return_gauge, expected_gauge)
        # Test 2
        input_string = ' 409025 '
        return_gauge = scenario_handling.extract_gauge_from_string(input_string)
        expected_gauge = '409025'
        self.assertEqual(return_gauge, expected_gauge)
        # Test 3
        input_string = '409025---999'
        return_gauge = scenario_handling.extract_gauge_from_string(input_string)
        expected_gauge = '409025'
        self.assertEqual(return_gauge, expected_gauge)

        
    def test_cleaner_IQQM_10000yr(self):
        '''
        1. Check date formatting and correct allocationo of gauge data to either flow/level dataframes
        '''
        # Set up input data and pass to test function:
        date_start = '0105-07-01'
        date_end = '9999-06-30'
        date_range = pd.period_range(date_start, date_end, freq = 'D')
        data_for_input_df = {'Date': date_range, '409025': [50]*3613709}
        input_df = pd.DataFrame(data_for_input_df)
        str_df = input_df.copy(deep=True)
        str_df['Date'] = str_df['Date'].astype('str')
        def add_0 (row):
            j = row.split('-')
            if len(j[0]) < 4:
                new_row = '0'+ row
            else:
                new_row = row
            return new_row
        str_df['Date'] = str_df['Date'].apply(add_0)
        str_df = str_df.set_index('Date')
        df_f, df_l = scenario_handling.cleaner_IQQM_10000yr(str_df)
        
        # Set up expected data and test:
        expected_df_flow = input_df.copy(deep=True)
        expected_df_flow = expected_df_flow.set_index('Date')
        expected_df_flow.columns = ['409025']
        expected_df_level = pd.DataFrame(index = expected_df_flow.index)
        
        assert_frame_equal(expected_df_level, df_l)
        assert_frame_equal(expected_df_flow, df_f)
        
        
    def test_cleaner_NSW(self):
        '''
        1. Test date formatting is applied correctly
        '''
        # Set up input data and pass to test function:
        model_metadata = data_inputs.get_NSW_codes()
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        date_strings = dates.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
        data_df = {'Date': date_strings,
                   'Gauge: YCB_410134_BillabongCreek@Darlot: Downstream Flow': [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10,
                   'Gauge: YCB_410016 Billabong Creek @ Jerilderie: Downstream Flow': [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10
                  }
        df = pd.DataFrame(data = data_df)
        df_clean = scenario_handling.cleaner_NSW(df)
        # Set up expected output data and test:
        data_expected_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                              'Gauge: YCB_410134_BillabongCreek@Darlot: Downstream Flow': [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10,
                              'Gauge: YCB_410016 Billabong Creek @ Jerilderie: Downstream Flow': [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10
                             }
        expected_df_F = pd.DataFrame(data_expected_df_F)
        expected_df_F = expected_df_F.set_index('Date')
        assert_frame_equal(df_clean, expected_df_F)
        
        
    def test_cleaner_MDBA(self):
        '''
        1. Test date formatting is applied correctly
        '''
        # Set up input data and pass to test function:
        data_df = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                   'EUSTDS-1-8': [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10,
                   'EUSTUS-35-8': [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10
                  }
        df = pd.DataFrame(data = data_df)
        df['Dy'], df['Mn'], df['Year'] = df['Date'].dt.day, df['Date'].dt.month, df['Date'].dt.year
        df = df.drop(['Date'], axis = 1)
        
        df_clean = scenario_handling.cleaner_MDBA(df)
        # Set up expected output data and test:
        data_expected_df = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                              'EUSTDS-1-8': [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10,
                              'EUSTUS-35-8': [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10
                             }
        expected_df = pd.DataFrame(data_expected_df)
        expected_df = expected_df.set_index('Date')
        
        assert_frame_equal(df_clean, expected_df)
        
        
    def test_build_NSW_columns(self):
        '''
        1. Check data columns are renamed based on header data
        '''
        # Load in test input data and pass to test function:
        input_file = 'unit_testing_files/NSW_source_res_test_file.csv'
        data, header = scenario_handling.unpack_model_file(input_file, 'Date', 'Field')
        data = scenario_handling.build_NSW_columns(data, header)
        # Load in test expected output data and test:
        expected_result = pd.read_csv('unit_testing_files/NSW_source_res_test_file_flow_result.csv')
        assert_frame_equal(data, expected_result)
        
        
    def test_build_MDBA_columns(self):
        '''
        1. Ensure data column names are correctly taken from the header data
        '''
        # Load input data and send to test function:
        input_file = 'unit_testing_files/MDBA_bigmod_test_file.csv'
        data, header = scenario_handling.unpack_model_file(input_file, 'Dy', 'Field')
        data = scenario_handling.build_MDBA_columns(data, header)
        data = data.astype({'Dy': 'float64'})
        # Load expected output data, format, and test:
        expected_result = pd.read_csv('unit_testing_files/MDBA_bigmod_test_file_flow_result.csv')
        expected_result = expected_result.astype({'Dy': 'float64'})
        assert_frame_equal(data, expected_result)        
    
    
    def test_unpack_model_file(self):
        '''
        1. Test MDBA style file ingestion
        2. Test NSW style file ingestion
        '''
        
        # Test 1
        # Load test data and expected output data
        file_to_pass = 'unit_testing_files/MDBA_bigmod_test_file.csv'
        expected_header = pd.read_csv('unit_testing_files/MDBA_bigmod_test_file_header_result.csv', dtype={'Site':'str', 'Measurand': 'str', 'Quality': 'str'})
        expected_flow = pd.read_csv('unit_testing_files/MDBA_bigmod_test_file_flow_result.csv', dtype={'Dy':'int', 'Mn': 'int', 'Year': 'int'})
        # Pass to test function and test
        flow, header = scenario_handling.unpack_model_file(file_to_pass, 'Dy', 'Field')
        assert_frame_equal(flow, expected_flow)
        assert_frame_equal(header, expected_header)
        #------------------------------------------
        # Test 2
        # Load test data and expected output data:
        file_to_pass = 'unit_testing_files/NSW_source_res_test_file.csv'
        expected_header = pd.read_csv('unit_testing_files/NSW_source_res_test_file_header_result.csv')
        expected_flow = pd.read_csv('unit_testing_files/NSW_source_res_test_file_flow_result.csv')
        expected_flow.columns = ['Date', '1>Data Sources>Data Sources@Climate Data@FAO56_res_csv@049050_SILO_FAO56.csv', '2>Data Sources>Data Sources@Climate Data@Mwet_res_csv@049050_SILO_Mwet.csv']
        # Pass to test function and test
        flow, header = scenario_handling.unpack_model_file(file_to_pass, 'Date', 'Field')  
        assert_frame_equal(flow, expected_flow)
        assert_frame_equal(header, expected_header)

        
    def test_unpack_IQQM_10000yr(self):
        '''
        1. Check gauge strings are pulled from column names and saved in place of the original string 
        '''
        file_to_pass ='unit_testing_files/NSW_10000yr_test_file.csv'
        flow = scenario_handling.unpack_IQQM_10000yr(file_to_pass)
        
        expected_flow = pd.read_csv('unit_testing_files/NSW_10000yr_test_file.csv', index_col = 'Date')
        expected_flow.columns = ['418013']
        
        assert_frame_equal(flow, expected_flow)
    
    
    def test_scenario_handler(self):
        '''things to test here:
        1. Ensure all parts of the function generate expected output
        '''
        # Testing the MDBA bigmod format:
        # Input params
        scenarios = {'Low_flow_EWRs_Bidgee_410007': 'unit_testing_files/Low_flow_EWRs_Bidgee_410007.csv'}
        model_format = 'Bigmod - MDBA'
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Pass to the function
        detailed, summary = scenario_handling.scenario_handler(scenarios, model_format, allowance, climate)
        # Expected output params
        expected_detailed_results = pd.read_csv('unit_testing_files/detailed_results_test.csv', index_col=0)
        expected_detailed_results.index = expected_detailed_results.index.astype('object')
        cols = expected_detailed_results.columns[expected_detailed_results.columns.str.contains('eventLength')]
        expected_detailed_results[cols] = expected_detailed_results[cols].astype('float64')
        for col in expected_detailed_results:
            if 'daysBetweenEvents' in col:
                for i, val in enumerate(expected_detailed_results[col]):
                    new = expected_detailed_results[col].iloc[i]
                    if new == '[]':
                        new_list = []
                    else:
                        new = re.sub('\[', '', new)
                        new = re.sub('\]', '', new)
                        new = new.split(',')
                        new_list = []
                        for days in new:
                            new_days = days.strip()
                            new_days = int(new_days)
                            new_list.append(new_days)

                    expected_detailed_results[col].iloc[i] = new_list
        # Test
        assert_frame_equal(detailed['Low_flow_EWRs_Bidgee_410007']['410007']['Upper Yanco Creek'], expected_detailed_results)
        
Test = test_scenario_handling()
suite = unittest.TestSuite()

suite.addTest(test_scenario_handling('test_match_MDBA_nodes'))
suite.addTest(test_scenario_handling('test_match_NSW_nodes'))
suite.addTest(test_scenario_handling('test_extract_gauge_from_string'))
suite.addTest(test_scenario_handling('test_cleaner_IQQM_10000yr'))
suite.addTest(test_scenario_handling('test_cleaner_NSW'))
suite.addTest(test_scenario_handling('test_cleaner_MDBA'))
suite.addTest(test_scenario_handling('test_build_NSW_columns'))
suite.addTest(test_scenario_handling('test_build_MDBA_columns'))
suite.addTest(test_scenario_handling('test_unpack_model_file'))
suite.addTest(test_scenario_handling('test_unpack_IQQM_10000yr'))
suite.addTest(test_scenario_handling('test_scenario_handler'))


unittest.TextTestRunner().run(suite)
        
        
        

E..

Attempted and failed to read in dates in format: dd/mm/yyyy, attempting to look for dates in format: yyyy-mm-dd
successfully read in data with yyyy-mm-dd formatting


....

Attempted and failed to read in dates in format: dd/mm/yyyy, attempting to look for dates in format: yyyy-mm-dd
successfully read in data with yyyy-mm-dd formatting


Evaluating scenarios:   0%|          | 0/1 [00:00<?, ?it/s]                                                                                                                                          
E
ERROR: test_match_MDBA_nodes (__main__.test_scenario_handling)
1. Ensure dataframe with flows and levels is split into two dataframes (one flow and one level dataframe)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_12590/383273231.py", line 7, in test_match_MDBA_nodes
    model_metadata = data_inputs.get_MDBA_codes()
  File "/home/pedro/code/ewr_fork/EWR_tool/py_ewr/data_inputs.py", line 159, in get_MDBA_codes
    metadata = pd.read_csv( BASE_PATH / 'model_metadata/SiteID_MDBA.csv', engine = 'python', dtype=str)
  File "/home/pedro/code/ewr_fork/.venv/lib/python3.8/site-packages/pandas/util/_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "/home/pedro/code/ewr_fork/.venv/lib/pyth

<unittest.runner.TextTestResult run=11 errors=2 failures=0>

### Testing the evaluate_EWRs.py module

#### Handling functions

In [15]:
class test_evaluate_EWRs_handling(unittest.TestCase):
    def test_ctf_handle(self):
        '''
        1. Ensure all parts of the function generate expected output
        '''
        # set up input data
        PU = 'PU_0000283'
        gauge = '410007'
        EWR = 'CF1'
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        data_for_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         gauge: [0]*1+[0]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*10+[0]*345+[0]*1+[0]*9 + [0]*5+[0]*351+[0]*10}
        df_F = pd.DataFrame(data = data_for_df_F)
        df_F = df_F.set_index('Date')
        PU_df = pd.DataFrame()
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Send input data to test function:
        PU_df, events = evaluate_EWRs.ctf_handle(PU, gauge, EWR, EWR_table, df_F, PU_df, allowance, climate)
        # Setting up expected output - PU_df
        data = {'CF1_eventYears': [0,0,0,1], 'CF1_numAchieved': [0,0,0,1], 'CF1_numEvents': [0,0,0,1], 'CF1_eventLength': [0.0,0.0,0.0,1461.0], 'CF1_totalEventDays': [0,0,0,1461], 'CF1_daysBetweenEvents': [[],[],[],[]], 
                'CF1_missingDays': [0,0,0,0], 'CF1_totalPossibleDays': [365,365,365,366]}
        index = [2012, 2013, 2014,2015]
        expected_PU_df = pd.DataFrame(index = index, data = data)
        expected_PU_df.index = expected_PU_df.index.astype('object')
#         print(PU_df.head())
#         print(expected_PU_df.head())        
        assert_frame_equal(PU_df, expected_PU_df)
        # Setting up expected output - events
        expected_events = {2012:[], 2013:[], 2014:[], 2015:[[0]*1461]}
        expected_events = tuple([expected_events])
        for index, tuple_ in enumerate(events):
            for year in events[index]:
                self.assertEqual(len(events[index][year]), len(expected_events[index][year]))
                for i, event in enumerate(events[index][year]):
                    self.assertListEqual(event, expected_events[index][year][i]) 
    
        
    def test_lowflow_handle(self):
        '''
        1. Ensure all parts of the function generate expected output
        '''
        # Set up input data
        PU = 'PU_0000283'
        gauge = '410007'
        EWR = 'BF1'
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        data_for_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         gauge: [0]*1+[250]*350+[0]*9+[0]*5 + [0]*360+[0]*5 + [0]*2+[250]*345+[0]*1+[250]*17 + [0]*5+[250]*351+[250]*10}
        df_F = pd.DataFrame(data = data_for_df_F)
        df_F = df_F.set_index('Date')
        PU_df = pd.DataFrame()
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Send input data to test function
        PU_df, events = evaluate_EWRs.lowflow_handle(PU, gauge, EWR, EWR_table, df_F, PU_df, allowance, climate)
        # Setting up expected output data - PU_df, and testing
        data = {'BF1_eventYears': [0,0,1,1], 'BF1_numAchieved': [0,0,1,1], 'BF1_numEvents': [0,0,1,1], 'BF1_eventLength': [350.0,0.0,181.0,361.0], 'BF1_totalEventDays': [350,0,362,361], 
                'BF1_daysBetweenEvents': [[],[],[381],[]],
                'BF1_missingDays': [0,0,0,0], 'BF1_totalPossibleDays': [365,365,365,366]}
        index = [2012, 2013, 2014,2015]
        expected_PU_df = pd.DataFrame(index = index, data = data)
        expected_PU_df.index = expected_PU_df.index.astype('object')
#         print(PU_df.head())
#         print(expected_PU_df.head())
        assert_frame_equal(PU_df, expected_PU_df)

        # Setting up expected output - events, and testing
        expected_events = {2012:[[250]*350], 2013:[], 2014:[[250]*345, [250]*17], 2015:[[250]*351+[250]*10]}
        expected_events = tuple([expected_events])
        for index, tuple_ in enumerate(events):
            for year in events[index]:
                self.assertEqual(len(events[index][year]), len(expected_events[index][year]))
                for i, event in enumerate(events[index][year]):
                    self.assertListEqual(event, expected_events[index][year][i])  
    
    def test_flow_handle(self):
        '''Things to calc in this function:
        1. Ensure all parts of the function generate expected output
        '''
        # Setting up input data
        PU = 'PU_0000283'
        gauge = '410007'
        EWR = 'SF1_S'
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        data_for_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         gauge: [0]*1+[250]*350+[450]*10+[0]*4 + [0]*360+[450]*5 + [450]*5+[250]*345+[0]*1+[450]*14 + [0]*5+[450]*10+[0]*1+[450]*10+[250]*330+[450]*10}
        df_F = pd.DataFrame(data = data_for_df_F)
        df_F = df_F.set_index('Date')
        PU_df = pd.DataFrame()
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Send input data to test function
        PU_df, events = evaluate_EWRs.flow_handle(PU, gauge, EWR, EWR_table, df_F, PU_df, allowance)
        # Setting up expected output - PU_df - and testing
        data = {'SF1_S_eventYears': [0,0,1,1], 'SF1_S_numAchieved': [0,0,1,1], 'SF1_S_numEvents': [1,0,2,3], 'SF1_S_eventLength': [10.0,0.0,12.0,10.0], 'SF1_S_totalEventDays': [10,0,24,30], 
                'SF1_S_daysBetweenEvents': [[],[],[],[]],
                'SF1_S_missingDays': [0,0,0,0], 'SF1_S_totalPossibleDays': [365,365,365,366]}
        index = [2012, 2013, 2014,2015]
        expected_PU_df = pd.DataFrame(index = index, data = data)
        expected_PU_df.index = expected_PU_df.index.astype('object')
#         print(PU_df.head())
#         print(expected_PU_df.head())
        assert_frame_equal(PU_df, expected_PU_df)
        # Setting up expected output - events - and testing
        expected_events = {2012:[[450]*10], 2013:[], 2014:[[450]*5+[450]*5, [450]*14], 2015:[[450]*10, [450]*10, [450]*10]}
        expected_events = tuple([expected_events])
        for index, tuple_ in enumerate(events):
            for year in events[index]:
                self.assertEqual(len(events[index][year]), len(expected_events[index][year]))
                for i, event in enumerate(events[index][year]):
                    self.assertListEqual(event, expected_events[index][year][i]) 
    
    def test_cumulative_handle(self):
        '''
        1. Ensure all parts of the function generate expected output
        '''
        # Set up input data
        PU = 'PU_0000040'
        gauge = '418068'
        EWR = 'OB3_S'
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        data_for_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         gauge: [0]*1+[0]*350+[10000]*1+[3000]*4+[0]*9 + [0]*360+[450]*3+[19000]*1+[1000]*1 + [450]*5+[250]*345+[0]*1+[0]*13+[5000]*1 + [5000]*4+[450]*10+[0]*2+[450]*10+[250]*330+[450]*10}
        df_F = pd.DataFrame(data = data_for_df_F)
        df_F = df_F.set_index('Date')
        PU_df = pd.DataFrame()
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Send input data to test function
        PU_df, events = evaluate_EWRs.cumulative_handle(PU, gauge, EWR, EWR_table, df_F, PU_df, allowance)
        # Setting up expected output - PU_df - and testing
        data = {'OB3_S_eventYears': [1,0,0,1], 'OB3_S_numAchieved': [1,0,0,1], 'OB3_S_numEvents': [1,0,0,1], 'OB3_S_eventLength': [5.0,0.0,0.0,5.0], 'OB3_S_totalEventDays': [5,0,0,5], 
                'OB3_S_daysBetweenEvents': [[],[],[],[]],
                'OB3_S_missingDays': [0,0,0,0], 'OB3_S_totalPossibleDays': [365,365,365,366]}
        index = [2012, 2013, 2014,2015]
        expected_PU_df = pd.DataFrame(index = index, data = data)
        expected_PU_df.index = expected_PU_df.index.astype('object')
#         print(PU_df.head())
#         print(expected_PU_df.head())
        assert_frame_equal(PU_df, expected_PU_df)
        # Setting up expected output - events - and testing 
        expected_events = {2012:[[10000]*1+[3000]*4], 2013:[], 2014:[], 2015:[[5000]*1+[5000]*4]}
        expected_events = tuple([expected_events])
        for index, tuple_ in enumerate(events):
            for year in events[index]:
                self.assertEqual(len(events[index][year]), len(expected_events[index][year]))
                for i, event in enumerate(events[index][year]):
                    self.assertListEqual(event, expected_events[index][year][i])

    def test_level_handle(self):
        '''
        1. Ensure all parts of the function generate expected output
        '''
        # Set up input data
        PU = 'PU_0000266'
        gauge = '425022'
        EWR = 'LLLF'
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        data_for_df_L = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         gauge: [0]*1+[0]*260+[56]*90+[0]*1+[0]*4+[0]*9 + [56]*45+[55.9]*1+[56]*45+[0]*269+[0]*3+[19000]*1+[1000]*1 + [0]*5+[0]*345+[0]*1+[0]*13+[56]*1 + [56]*89+[0]*4+[0]*10+[0]*3+[0]*10+[0]*230+[0]*20}
        df_L = pd.DataFrame(data = data_for_df_L)
        df_L = df_L.set_index('Date')
        PU_df = pd.DataFrame()
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Send input data to test function
        PU_df, events = evaluate_EWRs.level_handle(PU, gauge, EWR, EWR_table, df_L, PU_df, allowance)
        # Setting up expected output - PU_df and test
        data = {'LLLF_eventYears': [1,0,0,0], 'LLLF_numAchieved': [1,0,0,0], 'LLLF_numEvents': [1,0,0,0], 'LLLF_eventLength': [90.0,0.0,0.0,0], 'LLLF_totalEventDays': [90,0,0,0], 
                'LLLF_daysBetweenEvents': [[],[],[],[1110]],
                'LLLF_missingDays': [0,0,0,0], 'LLLF_totalPossibleDays': [365,365,365,366]}
        index = [2012, 2013, 2014,2015]
        expected_PU_df = pd.DataFrame(index = index, data = data)
        expected_PU_df.index = expected_PU_df.index.astype('object')
#         print(PU_df.head())
#         print(expected_PU_df.head())
        assert_frame_equal(PU_df, expected_PU_df)
        # Setting up expected output - events - and test
        expected_events = {2012:[[56]*90], 2013:[], 2014:[], 2015:[]}
        expected_events = tuple([expected_events])
        for index, tuple_ in enumerate(events):
            for year in events[index]:
                self.assertEqual(len(events[index][year]), len(expected_events[index][year]))
                for i, event in enumerate(events[index][year]):
                    self.assertListEqual(event, expected_events[index][year][i]) 
    
    def test_weirpool_handle(self):
        '''
        1. Ensure all parts of the function generate expected output
        '''
        # Set up input data
        PU = 'PU_0000260'
        gauge = '414203'
        wp_gauge = '414209'
        EWR = 'WP1'
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        # input data for df_L:
        levels = [47.3]*100 
        reduction_max = 0.04
        for i, level in enumerate(levels):
            levels[i] = levels[i-1]-reduction_max # Levels declining at acceptable rate
        exceeding_levels = [47.3]*100 
        reduction_max = 0.05
        for i, level in enumerate(exceeding_levels):
            exceeding_levels[i] = exceeding_levels[i-1]-reduction_max # Levels exceeding the acceptable rate: 
        data_for_df_L = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         wp_gauge: [0]*187+[47.3]*90+[0]*88 + [47.3]*90+[0]*275 + [0]*187+levels+[0]*78 + [0]*187+exceeding_levels+[0]*79}
        df_L = pd.DataFrame(data = data_for_df_L)
        df_L = df_L.set_index('Date')
        # input data for df_F:
        data_for_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         gauge: [0]*187+[2500]*90+[0]*88 + [2500]*90+[0]*275 + [0]*187+[2500]*100+[0]*78 + [0]*187+[2500]*100+[0]*79}
        df_F = pd.DataFrame(data = data_for_df_F)
        df_F = df_F.set_index('Date')
        PU_df = pd.DataFrame()
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Passing input data to test function
        PU_df, events = evaluate_EWRs.weirpool_handle(PU, gauge, EWR, EWR_table, df_F, df_L, PU_df, allowance)
        # Setting up expected output data - PU_df - and testing
        data = {'WP1_eventYears': [1,0,1,0], 'WP1_numAchieved': [1,0,1,0], 'WP1_numEvents': [1,0,1,0], 'WP1_eventLength': [90.0,0.0,90.0,0.0], 'WP1_totalEventDays': [90,0,90,0], 
                'WP1_daysBetweenEvents': [[],[],[],[]],
                'WP1_missingDays': [0,0,0,0], 'WP1_totalPossibleDays': [365,365,365,366]}
        index = [2012, 2013, 2014,2015]
        expected_PU_df = pd.DataFrame(index = index, data = data)
        expected_PU_df.index = expected_PU_df.index.astype('object')
#         print(PU_df.head())
#         print(expected_PU_df.head())
        assert_frame_equal(PU_df, expected_PU_df)
        # Setting up expected output - events - and testing
        expected_events = {2012:[[2500]*90], 2013:[], 2014:[[2500]*90], 2015:[]}
        expected_events = tuple([expected_events])
        for index, tuple_ in enumerate(events):
            for year in events[index]:
                self.assertEqual(len(events[index][year]), len(expected_events[index][year]))
                for i, event in enumerate(events[index][year]):
                    self.assertListEqual(event, expected_events[index][year][i])
    
    def test_nest_handle(self):
        '''
        1. Ensure all parts of the function generate expected output
        '''
        # Set up input data
        PU = 'PU_0000253'
        gauge = '409025'
        EWR = 'NestS1'
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        # input data up df_L:
        # flows declining at acceptable rate:
        acceptable_flows = [10000]*10
        reduction_max = 5.9
        for i, flow in enumerate(acceptable_flows):
            acceptable_flows[i] = acceptable_flows[i-1]-(reduction_max/100*acceptable_flows[i-1])
        acceptable_flows = acceptable_flows + [5300]*50
        # flows declining at unnacceptable rate:
        unnacceptable_flows = [10000]*10
        reduction_max = 7
        for i, flow in enumerate(unnacceptable_flows):
            unnacceptable_flows[i] = unnacceptable_flows[i-1]-(reduction_max/100*unnacceptable_flows[i-1])
        unnacceptable_flows = unnacceptable_flows + [5300]*50
        # flows declining at acceptable rate but going below the threshold
        threshold_flows = [10000]*10
        reduction_max = 6
        for i, flow in enumerate(threshold_flows):
            threshold_flows[i] = threshold_flows[i-1]-(reduction_max/100*threshold_flows[i-1])
        threshold_flows = threshold_flows + [5300]*50
        # input data for df_F:
        data_for_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         gauge: [0]*76+acceptable_flows+[0]*229 + [0]*76+unnacceptable_flows+[0]*229 + [0]*76+threshold_flows+[0]*229 + [0]*77+threshold_flows+[0]*229}
        df_F = pd.DataFrame(data = data_for_df_F)
        df_F = df_F.set_index('Date')
        df_L = pd.DataFrame()
        PU_df = pd.DataFrame()
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Pass input data to test function:
        PU_df, events = evaluate_EWRs.nest_handle(PU, gauge, EWR, EWR_table, df_F, df_L, PU_df, allowance)
        # Setting up expected output - PU_df - and testing
        data = {'NestS1_eventYears': [1,0,0,0], 'NestS1_numAchieved': [1,0,0,0], 'NestS1_numEvents': [1,0,0,0], 'NestS1_eventLength': [60.0,0.0,0.0,0.0], 'NestS1_totalEventDays': [60,0,0,0],
                'NestS1_daysBetweenEvents': [[],[],[],[1325]],
                'NestS1_missingDays': [0,0,0,0], 'NestS1_totalPossibleDays': [365,365,365,366]}
        index = [2012, 2013, 2014,2015]
        expected_PU_df = pd.DataFrame(index = index, data = data)
        expected_PU_df.index = expected_PU_df.index.astype('object')
#         print(PU_df.head())
#         print(expected_PU_df.head())
        assert_frame_equal(PU_df, expected_PU_df)
        # Setting up expected output - events - and testing
        expected_events = {2012:[acceptable_flows], 2013:[], 2014:[], 2015:[]}
        expected_events = tuple([expected_events])
        for index, tuple_ in enumerate(events):
            for year in events[index]:
                self.assertEqual(len(events[index][year]), len(expected_events[index][year]))
                for i, event in enumerate(events[index][year]):
                    self.assertListEqual(event, expected_events[index][year][i])
    
    def test_flow_handle_multi(self):
        '''
        1. Ensure all parts of the function generate expected output
        '''
        # Set up input data
        PU = 'PU_0000130'
        gauge1 = '421090'
        gauge2 = '421088'
        EWR = 'LF1'
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        data_for_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         gauge1: [0]*76+[1250]*5+[0]*229+[0]*55 + [0]*76+[0]*55+[0]*231+[1250]*3 + [1250]*3+[0]*76+[0]*50+[1250]*5+[0]*231 + [0]*77+[1250]*5+[0]*229+[0]*55,
                         gauge2: [0]*76+[1250]*5+[0]*229+[0]*55 + [0]*76+[0]*55+[0]*231+[1250]*3 + [1250]*3+[0]*76+[0]*50+[1250]*5+[0]*231 + [0]*76+[1250]*5+[0]*230+[0]*55
                        }
        df_F = pd.DataFrame(data = data_for_df_F)
        df_F = df_F.set_index('Date')
        df_L = pd.DataFrame()
        PU_df = pd.DataFrame()
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Send input data to test function
        PU_df, events = evaluate_EWRs.flow_handle_multi(PU, gauge1, EWR, EWR_table, df_F, PU_df, allowance)
        # Setting up expected output - PU_df - and testing
        data = {'LF1_eventYears': [1,0,1,0], 'LF1_numAchieved': [1,0,2,0], 'LF1_numEvents': [1,0,2,0], 'LF1_eventLength': [5.0,0.0,5.5,0.0], 'LF1_totalEventDays': [5,0,11,0],
                'LF1_daysBetweenEvents': [[],[],[],[]],
                'LF1_missingDays': [0,0,0,0], 'LF1_totalPossibleDays': [365,365,365,366]}
        index = [2012, 2013, 2014,2015]
        expected_PU_df = pd.DataFrame(index = index, data = data)
        expected_PU_df.index = expected_PU_df.index.astype('object')
#         print(PU_df.head())
#         print(expected_PU_df.head())
        assert_frame_equal(PU_df, expected_PU_df)    
        # Setting up expected output - events - and testing
        expected_events = {2012:[[2500]*5], 2013:[], 2014:[[2500]*6, [2500]*5], 2015:[]}
        expected_events = tuple([expected_events])
        for index, tuple_ in enumerate(events):
            for year in events[index]:
                self.assertEqual(len(events[index][year]), len(expected_events[index][year]))
                for i, event in enumerate(events[index][year]):
                    self.assertListEqual(event, expected_events[index][year][i])  
    
    def test_lowflow_handle_multi(self):
        '''
        1. Ensure all parts of the function generate expected output
        '''
        # Input data
        PU = 'PU_0000130'
        gauge1 = '421090'
        gauge2 = '421088'
        EWR = 'BF1'
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        data_for_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         gauge1: [50]*76+[1250]*5+[50]*229+[50]*15+[0]*40 + [50]*3+[0]*76+[0]*50+[0]*5+[0]*231 + [50]*75+[0]*50+[50]*230+[50]*10 + [0]*77+[50]*5+[0]*229+[50]*55,
                         gauge2: [50]*76+[1250]*5+[50]*229+[0]*40+[50]*15 + [50]*3+[0]*76+[0]*50+[0]*5+[0]*231 + [50]*75+[0]*50+[50]*230+[50]*10 + [0]*76+[50]*5+[0]*230+[50]*55
                        }
        df_F = pd.DataFrame(data = data_for_df_F)
        df_F = df_F.set_index('Date')
        df_L = pd.DataFrame()
        PU_df = pd.DataFrame()
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Pass input data to test function
        PU_df, events = evaluate_EWRs.lowflow_handle_multi(PU, gauge1, EWR, EWR_table, df_F, PU_df, allowance, climate)
        # Setting up expected output - PU_df - and testing
        data = {'BF1_eventYears': [0,0,1,0], 'BF1_numAchieved': [0,0,1,0], 'BF1_numEvents': [0,0,1,0], 'BF1_eventLength': [310.0,3.0,157.5,29.5], 'BF1_totalEventDays': [310,3,315,59],
                'BF1_daysBetweenEvents': [[],[362],[],[77, 230]],
                'BF1_missingDays': [0,0,0,0], 'BF1_totalPossibleDays': [365,365,365,366]}
        index = [2012, 2013, 2014,2015]
        expected_PU_df = pd.DataFrame(index = index, data = data)
        expected_PU_df.index = expected_PU_df.index.astype('object')
#         print(PU_df.head())
#         print(expected_PU_df.head())
        assert_frame_equal(PU_df, expected_PU_df)    
        # Setting up expected output - events - and testing
        expected_events = {2012:[[100]*76+[2500]*5+[100]*229], 2013:[[100]*3], 2014:[[100]*75, [100]*230+[100]*10], 2015:[[100]*4, [100]*55]}
        expected_events = tuple([expected_events])
        for index, tuple_ in enumerate(events):
            for year in events[index]:
                self.assertEqual(len(events[index][year]), len(expected_events[index][year]))
                for i, event in enumerate(events[index][year]):
                    self.assertListEqual(event, expected_events[index][year][i])
    
    def test_ctf_handle_multi(self):
        '''
        1. Ensure all parts of the function generate expected output
        '''
        # Set up the input data
        PU = 'PU_0000130'
        gauge1 = '421090'
        gauge2 = '421088'
        EWR = 'CF'
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        data_for_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         gauge1: [0]*1+[2]*350+[0]*9+[0]*5 + [2]*360+[0]*5 + [0]*10+[2]*345+[0]*1+[2]*9 + [0]*5+[0]*351+[0]*10,
                         gauge2: [0]*1+[2]*350+[0]*9+[0]*5 + [2]*360+[0]*5 + [0]*10+[2]*345+[0]*1+[2]*9 + [0]*5+[0]*351+[0]*10
                        }
        df_F = pd.DataFrame(data = data_for_df_F)
        df_F = df_F.set_index('Date')
        df_L = pd.DataFrame()
        PU_df = pd.DataFrame()
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Pass input data to the test function
        PU_df, events = evaluate_EWRs.ctf_handle_multi(PU, gauge1, EWR, EWR_table, df_F, PU_df, allowance, climate)
        # Setting up expected output - PU_df - and testing
        data = {'CF_eventYears': [1,0,1,1], 'CF_numAchieved': [2,0,2,1], 'CF_numEvents': [2,0,2,1], 'CF_eventLength': [7.5,0.0,8.0,366.0], 'CF_totalEventDays': [15,0,16,366],
                'CF_daysBetweenEvents': [[350],[360],[345,9],[]],
                'CF_missingDays': [0,0,0,0], 'CF_totalPossibleDays': [365,365,365,366]}
        index = [2012, 2013, 2014,2015]
        expected_PU_df = pd.DataFrame(index = index, data = data)
        expected_PU_df.index = expected_PU_df.index.astype('object')
#         print(PU_df.head())
#         print(expected_PU_df.head())
        assert_frame_equal(PU_df, expected_PU_df)    
        # Setting up expected output - events - and testing
        expected_events = {2012:[[0]*1, [0]*9+[0]*5], 2013:[], 2014:[[0]*15, [0]*1], 2015:[[0]*366]}
        expected_events = tuple([expected_events])
        for index, tuple_ in enumerate(events):
            for year in events[index]:
                self.assertEqual(len(events[index][year]), len(expected_events[index][year]))
                for i, event in enumerate(events[index][year]):
                    self.assertListEqual(event, expected_events[index][year][i])

    def test_cumulative_handle_multi(self):
        '''
        1. Ensure all parts of the function generate expected output
        '''
        # Set up input data
        PU = 'PU_0000132'
        gauge1 = '421090'
        gauge2 = '421088'
        EWR = 'OB/WS1_S'
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        data_for_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         gauge1: [0]*1+[0]*260+[334]*90+[0]*5+[0]*9 + [0]*310+[0]*3+[0]*1+[0]*1+[500]*50 + [500]*40+[0]*310+[0]*1+[0]*13+[0]*1 + [5000]*4+[500]*90+[500]*90+[450]*10+[0]*2+[450]*10+[250]*150+[450]*10,
                         gauge2: [0]*1+[0]*260+[334]*90+[0]*5+[0]*9 + [0]*310+[0]*3+[0]*1+[0]*1+[500]*50 + [500]*40+[0]*310+[0]*1+[0]*13+[0]*1 + [5000]*4+[500]*90+[500]*90+[450]*10+[0]*2+[450]*10+[250]*150+[450]*10
                        }
        df_F = pd.DataFrame(data = data_for_df_F)
        df_F = df_F.set_index('Date')
        df_L = pd.DataFrame()
        PU_df = pd.DataFrame()
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Pass input data to test function
        PU_df, events = evaluate_EWRs.cumulative_handle_multi(PU, gauge1, EWR, EWR_table, df_F, PU_df, allowance)
        # Setting up expected output - PU_df - and testing
        data = {'OB/WS1_S_eventYears': [1,1,1,1], 'OB/WS1_S_numAchieved': [1,1,1,2], 'OB/WS1_S_numEvents': [1,1,1,2], 'OB/WS1_S_eventLength': [90,90.0,90.0,90.0], 'OB/WS1_S_totalEventDays': [90,90,90,180],
                'OB/WS1_S_daysBetweenEvents': [[],[],[],[]],
                'OB/WS1_S_missingDays': [0,0,0,0], 'OB/WS1_S_totalPossibleDays': [365,365,365,366]}
        index = [2012, 2013, 2014,2015]
        expected_PU_df = pd.DataFrame(index = index, data = data)
        expected_PU_df.index = expected_PU_df.index.astype('object')
#         print(PU_df.head())
#         print(expected_PU_df.head())
        assert_frame_equal(PU_df, expected_PU_df)   
        # Setting up expected output - events - and testing
        expected_events = {2012:[[334*2]*90], 2013:[[0]*30+[500*2]*60], 2014:[[0]*66+[5000*2]*4+[500*2]*20], 2015:[[500*2]*90, [500*2]*70+[450*2]*10+[0]*2+[450*2]*8]}
        expected_events = tuple([expected_events])
        for index, tuple_ in enumerate(events):
            for year in events[index]:
                self.assertEqual(len(events[index][year]), len(expected_events[index][year]))
                for i, event in enumerate(events[index][year]):
                    self.assertListEqual(event, expected_events[index][year][i])     
    
    def test_flow_handle_sim(self):
        '''
        1. Ensure all parts of the function generate expected output
        '''
        # Set up input data
        PU = 'PU_0000131'
        gauge1 = '421090'
        gauge2 = '421022'
        EWR = 'LF1_S'
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        data_for_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         gauge1: [0]*76+[1000]*5+[0]*229+[0]*55 + [0]*76+[0]*55+[0]*231+[1000]*3 + [1000]*3+[0]*76+[0]*50+[1000]*5+[0]*231 + [0]*77+[1000]*5+[0]*229+[0]*55,
                         gauge2: [0]*76+[1000]*5+[0]*229+[0]*55 + [0]*76+[0]*55+[0]*231+[1000]*3 + [1000]*3+[0]*76+[0]*50+[1000]*5+[0]*231 + [0]*76+[1000]*5+[0]*230+[0]*55
                        }
        df_F = pd.DataFrame(data = data_for_df_F)
        df_F = df_F.set_index('Date')
        df_L = pd.DataFrame()
        PU_df = pd.DataFrame()
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Pass input data to test function
        PU_df, events = evaluate_EWRs.flow_handle_sim(PU, gauge1, EWR, EWR_table, df_F, PU_df, allowance)
        # Setting up expected output - PU_df - and testing
        data = {'LF1_S_eventYears': [1,0,1,0], 'LF1_S_numAchieved': [1,0,2,0], 'LF1_S_numEvents': [1,0,2,0], 'LF1_S_eventLength': [5.0,0.0,5.5,0.0], 'LF1_S_totalEventDays': [5,0,11,0],
                'LF1_S_daysBetweenEvents': [[],[],[],[]],
                'LF1_S_missingDays': [0,0,0,0], 'LF1_S_totalPossibleDays': [365,365,365,366]}
        index = [2012, 2013, 2014,2015]
        expected_PU_df = pd.DataFrame(index = index, data = data)
        expected_PU_df.index = expected_PU_df.index.astype('object')
#         print(PU_df.head())
#         print(expected_PU_df.head())
        assert_frame_equal(PU_df, expected_PU_df)
        # Setting up expected output - events - and testing
        expected_events = {2012:[[1000]*5], 2013:[], 2014:[[1000]*6, [1000]*5], 2015:[]}
        expected_events = tuple([expected_events])
        for index, tuple_ in enumerate(events):
            for year in events[index]:
                self.assertEqual(len(events[index][year]), len(expected_events[index][year]))
                for i, event in enumerate(events[index][year]):
                    self.assertListEqual(event, expected_events[index][year][i])  
    
    def test_lowflow_handle_sim(self):
        '''
        1. Ensure all parts of the function generate expected output
        '''
        # Set up input data
        PU = 'PU_0000131'
        gauge1 = '421090'
        gauge2 = '421022'
        EWR = 'BF1'
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        data_for_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         gauge1: [0]*76+[65]*280+[0]*9 + [0]*76+[0]*9+[65]*280 + [0]*80+[0]*9+[65]*276 + [65]*270+[0]*76+[0]*14+[65]*6,
                         gauge2: [0]*76+[65]*280+[0]*9 + [65]*280+[0]*76+[0]*9 + [0]*80+[0]*9+[65]*276 + [65]*270+[0]*76+[0]*14+[65]*6
                        }
        df_F = pd.DataFrame(data = data_for_df_F)
        df_F = df_F.set_index('Date')
        df_L = pd.DataFrame()
        PU_df = pd.DataFrame()
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Pass input data to test function
        PU_df, events = evaluate_EWRs.lowflow_handle_sim(PU, gauge1, EWR, EWR_table, df_F, PU_df, allowance, climate)
        # Setting up expected output - PU_df - and test
        # Note the floats that get returned in the total event days series. This is because the totals of the two series are averaged.
        data = {'BF1_eventYears': [1,1,1,0], 'BF1_numAchieved': [1,1,1,0], 'BF1_numEvents': [1,1,1,0], 'BF1_eventLength': [280.0,280.0,276.0,138.0], 'BF1_totalEventDays': [280.0,280.0,276.0,276.0],
                'BF1_daysBetweenEvents': [[76],[94],[89],[90]],
                'BF1_missingDays': [0,0,0,0], 'BF1_totalPossibleDays': [365,365,365,366]}
        index = [2012, 2013, 2014,2015]
        expected_PU_df = pd.DataFrame(index = index, data = data)
        expected_PU_df.index = expected_PU_df.index.astype('object')
        assert_frame_equal(PU_df, expected_PU_df)
        # Setting up expected output - events - and testing
        expected_events1 = {2012:[[65]*280], 2013:[[65]*280], 2014:[[65]*276], 2015:[[65]*270, [65]*6]}
        expected_events2 = {2012:[[65]*280], 2013:[[65]*280], 2014:[[65]*276], 2015:[[65]*270, [65]*6]}
        expected_events = tuple([expected_events1, expected_events2])
        for index, tuple_ in enumerate(events):
            for year in events[index]:
                self.assertEqual(len(events[index][year]), len(expected_events[index][year]))
                for i, event in enumerate(events[index][year]):
                    self.assertListEqual(event, expected_events[index][year][i]) 

    def test_ctf_handle_sim(self):
        '''
        1. Ensure all parts of the function generate expected output
        '''
        # Set up input data
        PU = 'PU_0000131'
        gauge1 = '421090'
        gauge2 = '421022'
        EWR = 'CF'
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        data_for_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         gauge1: [5]*123+[0]*5+[5]*232+[0]*5 + [0]*1+[5]*123+[0]*3+[5]*233+[0]*3+[5]*2 + [5]*123+[0]*5+[5]*232+[0]*5 + [5]*123+[0]*5+[5]*233+[5]*5,
                         gauge2: [5]*123+[0]*5+[5]*232+[0]*5 + [0]*1+[5]*123+[0]*3+[5]*233+[0]*3+[5]*2 + [5]*123+[0]*5+[5]*232+[0]*5 + [5]*123+[5]*5+[5]*233+[0]*5
                        }
        df_F = pd.DataFrame(data = data_for_df_F)
        df_F = df_F.set_index('Date')
        df_L = pd.DataFrame()
        PU_df = pd.DataFrame()
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Pass input data to test function
        PU_df, events = evaluate_EWRs.ctf_handle_sim(PU, gauge1, EWR, EWR_table, df_F, PU_df, allowance, climate)
        # Setting up expected output - PU_df - and test
        # Note the floats that get returned in the total event days series. This is because the totals of the two series are averaged.
        data = {'CF_eventYears': [1,0,0,1], 'CF_numAchieved': [2,0,0,1], 'CF_numEvents': [2,0,0,1], 'CF_eventLength': [5.0,3.0,5.0,5.0], 'CF_totalEventDays': [10.0,6.0,10.0,5.0],
                'CF_daysBetweenEvents': [[123, 232],[124,233],[125,232],[123,238]],
                'CF_missingDays': [0,0,0,0], 'CF_totalPossibleDays': [365,365,365,366]}
        index = [2012, 2013, 2014,2015]
        expected_PU_df = pd.DataFrame(index = index, data = data)
        expected_PU_df.index = expected_PU_df.index.astype('object')
        assert_frame_equal(PU_df, expected_PU_df) 
        # Setting up expected output - events - and test
        expected_events1 = {2012:[[0]*5, [0]*5], 2013:[[0]*3, [0]*3], 2014:[[0]*5, [0]*5], 2015:[[0]*5]}
        expected_events2 = {2012:[[0]*5, [0]*5], 2013:[[0]*3, [0]*3], 2014:[[0]*5, [0]*5], 2015:[[0]*5]}
        expected_events = tuple([expected_events1, expected_events2])
        for index, tuple_ in enumerate(events):
            for year in events[index]:
                self.assertEqual(len(events[index][year]), len(expected_events[index][year]))
                for i, event in enumerate(events[index][year]):
                    self.assertListEqual(event, expected_events[index][year][i])
   
    def test_complex_handle(self):
        '''
        1. Ensure all parts of the function generate expected output for OB2
        2. Ensure all parts of the function generate expected output for OB3
        '''
        # Test 1
        # Set up input data
        PU = 'PU_0000253'
        gauge = '409025'
        EWR = 'OB2a_S'
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        data_for_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         gauge: [19000]*45+[9000]*105+[0]*215 + [15000]*45+[0]*8+[9000]*105+[0]*207 + [15000]*15+[0]*6+[15000]*15+[0]*6+[15000]*15+[0]*6+[9000]*55+[0]*6+[9000]*50+[0]*150+[16000]*41 + \
                                         [0]*6+[16000]*4+[0]*6+[9000]*105+[0]*95+[18000]*45+[9000]*105,
                        }
        df_F = pd.DataFrame(data = data_for_df_F)
        df_F = df_F.set_index('Date')
        df_L = pd.DataFrame()
        PU_df = pd.DataFrame()
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Pass input data to test function
        PU_df, events = evaluate_EWRs.complex_handle(PU, gauge, EWR, EWR_table, df_F, PU_df, allowance)
        # Setting up expected output - PU_df - and testing
        data = {'OB2a_S_eventYears': [1,0,1,1], 'OB2a_S_numAchieved': [1,0,1,2], 'OB2a_S_numEvents': [1,0,1,2], 'OB2a_S_eventLength': [150.0,0.0,150.0,150.0], 'OB2a_S_totalEventDays': [150,0,150,300],
                'OB2a_S_daysBetweenEvents': [[],[],[],[]],
                'OB2a_S_missingDays': [0,0,0,0], 'OB2a_S_totalPossibleDays': [365,365,365,366]}
        index = [2012, 2013, 2014,2015]
        expected_PU_df = pd.DataFrame(index = index, data = data)
        expected_PU_df.index = expected_PU_df.index.astype('object')
        assert_frame_equal(PU_df, expected_PU_df) 
        # Setting up expected output - events - and testing
        expected_events = {2012:[[19000]*45+[9000]*105], 2013:[], 2014:[[15000]*15+[15000]*15+[15000]*15+[9000]*55+[9000]*50], 2015:[[16000]*41+[16000]*4+[9000]*105, [18000]*45+[9000]*105]}
        expected_events = tuple([expected_events])
        for index, tuple_ in enumerate(events):
            for year in events[index]:
                self.assertEqual(len(events[index][year]), len(expected_events[index][year]))
                for i, event in enumerate(events[index][year]):
                    self.assertListEqual(event, expected_events[index][year][i])
        #-------------------------------------------------------------------------------------
        # Test 2
        # Set up input data
        PU = 'PU_0000253'
        gauge = '409025'
        EWR = 'OB3a_S'
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        data_for_df_F = {'Date': pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d')),
                         gauge:  [25000]*21+[15000]*90+[0]*254 + [25000]*21+[0]*8+[15000]*90+[0]*246 + [15000]*90+[0]*6+[25000]*21+[0]*8+[25000]*15+[0]*6+[25000]*6+[0]*6+[15000]*80+[0]*6+[15000]*10+[0]*94+[15000]*17 + \
                                         [0]*6+[15000]*73+[0]*6+[25000]*1+[25000]*20+[0]*149+[25000]*21+[15000]*90
                        }
        df_F = pd.DataFrame(data = data_for_df_F)
        df_F = df_F.set_index('Date')
        df_L = pd.DataFrame()
        PU_df = pd.DataFrame()
        allowance = {'minThreshold': 1.0, 'maxThreshold': 1.0, 'duration': 1.0, 'drawdown': 1.0}
        climate = 'Standard - 1911 to 2018 climate categorisation'
        # Pass input data to test function
        PU_df, events = evaluate_EWRs.complex_handle(PU, gauge, EWR, EWR_table, df_F, PU_df, allowance)
        # Setting up expected output - PU_df - and testing
        data = {'OB3a_S_eventYears': [1,0,1,1], 'OB3a_S_numAchieved': [1,0,2,2], 'OB3a_S_numEvents': [1,0,2,2], 'OB3a_S_eventLength': [111.0,0.0,111.0,111.0], 'OB3a_S_totalEventDays': [111,0,222,222],
                'OB3a_S_daysBetweenEvents': [[],[],[],[]],
                'OB3a_S_missingDays': [0,0,0,0], 'OB3a_S_totalPossibleDays': [365,365,365,366]}
        index = [2012, 2013, 2014,2015]
        expected_PU_df = pd.DataFrame(index = index, data = data)
        expected_PU_df.index = expected_PU_df.index.astype('object')
        assert_frame_equal(PU_df, expected_PU_df) 
        # Setting up expected output - events - and testing
        expected_events = {2012:[[25000]*21+[15000]*90], 2013:[], 2014:[[15000]*90+[25000]*21, [25000]*15+[25000]*6+[15000]*80+[15000]*10], 2015:[[15000]*17+[15000]*73+[25000]*21, [25000]*21+[15000]*90]}
        expected_events = tuple([expected_events])
        for index, tuple_ in enumerate(events):
            for year in events[index]:
                self.assertEqual(len(events[index][year]), len(expected_events[index][year]))
                for i, event in enumerate(events[index][year]):
                    self.assertListEqual(event, expected_events[index][year][i])

Test = test_evaluate_EWRs_handling()
suite = unittest.TestSuite()

suite.addTest(test_evaluate_EWRs_handling('test_ctf_handle'))
suite.addTest(test_evaluate_EWRs_handling('test_lowflow_handle'))
suite.addTest(test_evaluate_EWRs_handling('test_flow_handle'))
suite.addTest(test_evaluate_EWRs_handling('test_cumulative_handle'))
suite.addTest(test_evaluate_EWRs_handling('test_level_handle'))
suite.addTest(test_evaluate_EWRs_handling('test_weirpool_handle'))
suite.addTest(test_evaluate_EWRs_handling('test_nest_handle'))
suite.addTest(test_evaluate_EWRs_handling('test_flow_handle_multi'))
suite.addTest(test_evaluate_EWRs_handling('test_lowflow_handle_multi'))
suite.addTest(test_evaluate_EWRs_handling('test_ctf_handle_multi'))
suite.addTest(test_evaluate_EWRs_handling('test_cumulative_handle_multi'))
suite.addTest(test_evaluate_EWRs_handling('test_flow_handle_sim'))
suite.addTest(test_evaluate_EWRs_handling('test_lowflow_handle_sim'))
suite.addTest(test_evaluate_EWRs_handling('test_ctf_handle_sim'))
suite.addTest(test_evaluate_EWRs_handling('test_complex_handle'))

unittest.TextTestRunner().run(suite)

.....E...FEEEE.
ERROR: test_weirpool_handle (__main__.test_evaluate_EWRs_handling)
1. Ensure all parts of the function generate expected output
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_12590/129174869.py", line 220, in test_weirpool_handle
    PU_df, events = evaluate_EWRs.weirpool_handle(PU, gauge, EWR, EWR_table, df_F, df_L, PU_df, allowance)
  File "/home/pedro/code/ewr_fork/EWR_tool/py_ewr/evaluate_EWRs.py", line 313, in weirpool_handle
    EWR_info = get_EWRs(PU, gauge, EWR, EWR_table, allowance, pull)
  File "/home/pedro/code/ewr_fork/EWR_tool/py_ewr/evaluate_EWRs.py", line 36, in get_EWRs
    start_date = str(component_pull(EWR_table, gauge, PU, EWR, 'start month'))
  File "/home/pedro/code/ewr_fork/EWR_tool/py_ewr/evaluate_EWRs.py", line 17, in component_pull
    component = list(EWR_table[((EWR_table['gauge'] == gauge) &
IndexError: list index out of range

ERROR: test_cumulative_handle_mul

<unittest.runner.TextTestResult run=15 errors=5 failures=1>

#### Calculation functions + remaining functions in evaluate_EWRs module

In [16]:
class test_evaluate_EWRs(unittest.TestCase):
    def test_component_pull(self):
        '''
        1. Test correct value is pulled from EWR dataset
        '''
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        gauge = '409025'
        PU = 'PU_0000253'
        EWR = 'SF1_P'
        component = 'duration'
        self.assertEqual(evaluate_EWRs.component_pull(EWR_table, gauge, PU, EWR, component), '10')
        
    def test_apply_correction(self):
        '''
        1. Test function that applies relaxation to parts of indicator
        '''
        info = 960
        correction = 1.2
        self.assertEqual(evaluate_EWRs.apply_correction(info, correction), 1152.0)
        
    def test_get_EWRs(self):
        '''
        1. Ensure requested parts of EWR are returned
        '''
        EWR_table, bad_EWRs = data_inputs.get_EWR_table()
        PU = 'PU_0000283'
        gauge = '410007'
        EWR = 'SF1_P'
        minThreshold_tolerance = (100 - 0)/100
        maxThreshold_tolerance = (100 + 0)/100
        duration_tolerance = (100 - 0)/100
        drawdown_tolerance = (100 - 0)/100
        allowance ={'minThreshold': minThreshold_tolerance, 'maxThreshold': maxThreshold_tolerance,
                        'duration': duration_tolerance, 'drawdown': drawdown_tolerance}
        components = ['SM', 'EM']
        
        expected = {'gauge': '410007', 'planning_unit': 'PU_0000283', 'EWR_code': 'SF1_P', 'start_day': None, 'start_month': 10, 'end_day': None, 'end_month':4}
        self.assertEqual(evaluate_EWRs.get_EWRs(PU, gauge, EWR, EWR_table, allowance, components), expected)
    
    def test_mask_dates(self):
        '''
        This testing function will also be testing the functions get_month_mask, get_day_month_mask, and get_day_month_mask
        1. Testing for no filtering (all year round EWR requirement)
        2. Testing for a month subset
        3. Testing for water year crossover EWR requirement
        4. Testing for start day and end day within same month inclusion in the EWR requirement
        5. Testing for start day and end day within different months inclusion in the EWR requirement:
        '''
        #------------ Dataframe to be passed to all testing functions here ----------#
        
        data = {'409102': list(range(0,3650,10)), '425012': list(range(0,3650,10))}
        index = pd.date_range(start='1/1/2019', end='31/12/2019')
        df = pd.DataFrame(data = data, index = index)
        
        #----------------------------------------------------------------------------#
        # Test 1
        EWR_info = {'start_day': None, 'end_day': None, 'start_month': 7, 'end_month': 6}
        masked_7to6 = set(pd.date_range(start='1/1/2019', end='31/12/2019'))
        self.assertEqual(evaluate_EWRs.mask_dates(EWR_info, df), masked_7to6)
        #-----------------------------------------------------------------------
        # Test 2
        EWR_info = {'start_day': None, 'end_day': None, 'start_month': 7, 'end_month': 9}
        masked_7to9 = set(pd.date_range(start='2019-07-01', end='2019-09-30'))
        self.assertEqual(evaluate_EWRs.mask_dates(EWR_info, df), masked_7to9)
        #-----------------------------------------------------------------------
        # Test 3
        EWR_info = {'start_day': None, 'end_day': None, 'start_month': 6, 'end_month': 8}
        masked_6to8 = set(pd.date_range(start='2019-06-01', end='2019-08-31'))
        self.assertEqual(evaluate_EWRs.mask_dates(EWR_info, df), masked_6to8)
        #-----------------------------------------------------------------------
        # Test 4
        EWR_info = {'start_day': 12, 'end_day': 28, 'start_month': 6, 'end_month': 6}
        masked_612to628 = set(pd.date_range(start='2019-06-12', end='2019-06-28'))
        self.assertEqual(evaluate_EWRs.mask_dates(EWR_info, df), masked_612to628)
        #-----------------------------------------------------------------------
        # Test 5
        EWR_info = {'start_day': 12, 'end_day': 18, 'start_month': 8, 'end_month': 9}
        masked_812to918 = set(pd.date_range(start='2019-08-12', end='2019-09-18'))
        self.assertEqual(evaluate_EWRs.mask_dates(EWR_info, df), masked_812to918)
    def test_wateryear_daily(self):
        '''
        1. Testing non standard water year part of function
        2. Testing standard water year part of function
        '''
        # Test 1
        EWR_info = {'start_day': None, 'end_day': None, 'start_month': 5, 'end_month': 8}
        data = {'409102': list(range(0,3650,10)), '425012': list(range(0,3650,10))}
        index = pd.date_range(start='1/1/2019', end='31/12/2019')
        df = pd.DataFrame(data = data, index = index)
        expected_2018 = [2018]*120
        expected_2019 = [2019]*245
        expected_array = np.array(expected_2018 + expected_2019)
        array = evaluate_EWRs.wateryear_daily(df, EWR_info)
        self.assertTrue(np.array_equal(array, expected_array))
        #-------------------------------------------------------
        # Test 2
        EWR_info = {'start_day': None, 'end_day': None, 'start_month': 7, 'end_month': 9}
        data = {'409102': list(range(0,3650,10)), '425012': list(range(0,3650,10))}
        index = pd.date_range(start='1/1/2019', end='31/12/2019')
        df = pd.DataFrame(data = data, index = index)
        expected_2018 = [2018]*181
        expected_2019 = [2019]*184
        expected_array = np.array(expected_2018 + expected_2019)
        array = evaluate_EWRs.wateryear_daily(df, EWR_info)
        self.assertTrue(np.array_equal(array, expected_array))
    def test_which_wateryear(self):
        '''
        1. Testing for when there is an equal portion of the event falling in two water years
        2. Testing for when there is a non-equal portion of the event falling within each water year
        '''
        # Test 1
        i = 45
        event = [10]*10
        water_years = np.array([2018]*40+[2019]*10)
        water_year = evaluate_EWRs.which_water_year(i, len(event), water_years)
        self.assertEqual(water_year, 2019)
        #----------------------------------
        # Test 2
        i = 42
        event = [10]*9
        water_years = np.array([2018]*40+[2019]*10)
        water_year = evaluate_EWRs.which_water_year(i, len(event), water_years)
        self.assertEqual(water_year, 2018)
    def test_get_duration(self):
        '''
        1. Test for very dry duration for EWR where a very dry duration requirement exists
        2. Test for very dry duration for EWR where a very dry duration requirement does not exists
        '''
        # Test 1
        EWR_info = {'duration_VD': 2, 'duration': 5}
        duration = evaluate_EWRs.get_duration('Very Dry', EWR_info)
        self.assertEqual(duration, 2)
        #-----------------------------
        # Test 2
        EWR_info = {'duration_VD': None, 'duration': 5}
        duration = evaluate_EWRs.get_duration('Very Dry', EWR_info)
        self.assertEqual(duration, 5)
    def test_construct_event_dict(self):
        '''
        1. Test event dictionary with correct keys and values are returned
        '''
        water_years = [2018]*365+[2019]*365
        all_events = evaluate_EWRs.construct_event_dict(water_years)
        expected_result = {2018:[], 2019:[]}
        self.assertEqual(all_events, expected_result)
    def test_get_days_between(self):
        '''
        1. Testing low flow with more than 1 year interevent requirement
        2. Testing low flow with less than 1 year of interevent requirement
        3. Testing non low flow EWR with more than 1 year requirement
        4. Testing for EWR with only a subset of the water year available
        '''
        # Test 1
        no_events = {2012: [[735], [50], [2]], 2013: [[35], [50], [365]],
                     2014: [[35], [50], [2]], 2015: [[35], [280], [2]]}
        years_with_events = [0,0,0,1] # This will be used in the calculation
        EWR = 'BF'
        EWR_info = {'max_inter-event': 2}
        unique_water_years = [2012, 2013, 2014, 2015]
        water_years = [2012]*365+[2013]*365+[2014]*365+[2015]*365
        days_between = evaluate_EWRs.get_days_between(years_with_events, no_events, EWR, EWR_info, unique_water_years, water_years)
        expected_days_between = [[], [], [], [1095]]
        for i, v in enumerate(days_between):
            self.assertListEqual(days_between[i], expected_days_between[i])
        #--------------------------------------------------------------------
        # Test 2
        no_events = {2012: [[35], [50], [2]], 2013: [[35], [50], [2]],
                     2014: [[35], [50], [2]], 2015: [[35], [50], [2]]}
        years_with_events = [0,0,0,1] #This will be ignored in this calculation
        EWR = 'BF'
        EWR_info = {'max_inter-event': 0.04} # Equates to about 14-15 days
        unique_water_years = [2012, 2013, 2014, 2015]
        water_years = [2012]*365+[2013]*365+[2014]*365+[2015]*365
        days_between = evaluate_EWRs.get_days_between(years_with_events, no_events, EWR, EWR_info, unique_water_years, water_years)
        expected_days_between = [[35, 50], [35, 50], [35, 50], [35, 50]]
        for i, v in enumerate(days_between):
            self.assertListEqual(days_between[i], expected_days_between[i])
        #--------------------------------------------------------------------
        # Test 3
        no_events = {2012: [[35], [50], [2]], 2013: [[35], [50], [2]],
                     2014: [[35], [50], [2]], 2015: [[735], [2]]}
        years_with_events = [0,1,0,0] #This will be ignored in this calculation
        EWR = 'LF'
        EWR_info = {'max_inter-event': 2}
        unique_water_years = [2012, 2013, 2014, 2015]
        water_years = [2012]*365+[2013]*365+[2014]*365+[2015]*365
        days_between = evaluate_EWRs.get_days_between(years_with_events, no_events, EWR, EWR_info, unique_water_years, water_years)
        expected_days_between = [[], [], [], [735]]
        for i, v in enumerate(days_between):
            self.assertListEqual(days_between[i], expected_days_between[i])
        #--------------------------------------------------------------------
        # Test 4
        no_events = {2012: [[35], [122], [2]], 2013: [[35], [50], [2]],
                     2014: [[35], [50], [2]], 2015: [[730], [50], [121]]}
        years_with_events = [0,0,0,1] #This will be ignored in this calculation
        EWR = 'LF'
        EWR_info = {'max_inter-event': 2}
        unique_water_years = [2012, 2013, 2014, 2015]
        water_years = [2012]*365+[2013]*365+[2014]*365+[2015]*365
        days_between = evaluate_EWRs.get_days_between(years_with_events, no_events, EWR, EWR_info, unique_water_years, water_years)
        expected_days_between = [[], [], [], [730]]
        for i, v in enumerate(days_between):          
            self.assertListEqual(days_between[i], expected_days_between[i])
    def test_get_event_years(self):
        '''
        Year 1: check 1 is returned when there are 3 events with 2 required per year
        Year 2: check 0 is returned when there is 1 event with 2 required per year
        Year 3: check 1 is returned when there are 4 events with 2 required per year
        Year 4: check 0 is returned when there are 0 events with 2 required per year
        '''
        EWR_info = {'events_per_year': 2}
        events = {2012: [[5]*5, [10]*5, [20*8]], 2013: [[50]*20],
                  2014: [[5]*5, [10]*5, [20*8], [20*8]], 2015: []}
        unique_water_years = [2012, 2013, 2014, 2015]
        durations = [5,5,5,5]
        min_events = [5,5,5,5]
        event_years = evaluate_EWRs.get_event_years(EWR_info, events, unique_water_years, durations, min_events)
        expected_event_years = [1,0,1,0]
        self.assertEqual(event_years, expected_event_years)
    def test_get_achievements(self):
        '''
        1. Testing 1 event per year requirement with four unique events per year ranges
        2. Testing 2 events per year requirement with four unique events per year ranges
        '''
        EWR_info = {'events_per_year': 1}
        events = {2012: [[5]*5, [10]*5, [20]*8], 2013: [[50]*20],
                  2014: [[5]*5, [10]*5, [20]*8, [20]*8], 2015: []}
        unique_water_years = [2012, 2013, 2014, 2015]
        durations = [5,5,5,5]
        min_events = [5,5,5,5]
        num_events = evaluate_EWRs.get_achievements(EWR_info, events, unique_water_years, durations, min_events)
        expected_num_events = [3,1,4,0]
        self.assertEqual(num_events, expected_num_events)
        #-------------------------------------------------
        # Test 2
        EWR_info = {'events_per_year': 2}
        events = {2012: [[5]*5, [10]*5, [20]*8], 2013: [[50]*20],
                  2014: [[5]*5, [10]*5, [20]*8, [20]*8], 2015: []}
        unique_water_years = [2012, 2013, 2014, 2015]
        durations = [5,5,5,5]
        min_events = [5,5,5,5]
        num_events = evaluate_EWRs.get_achievements(EWR_info, events, unique_water_years, durations, min_events)
        expected_num_events = [1,0,2,0]
        self.assertEqual(num_events, expected_num_events)
    def test_get_number_events(self):
        '''
        1. Testing 1 event per year requirement with four unique events per year ranges
        2. Testing 2 events per year requirement with four unique events per year ranges
        '''
        # Test 1
        EWR_info = {'events_per_year': 1}
        events = {2012: [[5]*5, [10]*5, [20]*8], 2013: [[50]*20],
                  2014: [[5]*5, [10]*5, [20]*8, [20]*8], 2015: []}
        unique_water_years = [2012, 2013, 2014, 2015]
        durations = [5,5,5,5]
        min_events = [5,5,5,5]
        num_events = evaluate_EWRs.get_number_events(EWR_info, events, unique_water_years, durations, min_events)
        expected_num_events = [3,1,4,0]
        self.assertEqual(num_events, expected_num_events)
        #--------------------------------------------------
        # Test 2
        EWR_info = {'events_per_year': 2}
        events = {2012: [[5]*5, [10]*5, [20]*8], 2013: [[50]*20],
                  2014: [[5]*5, [10]*5, [20]*8, [20]*8], 2015: []}
        unique_water_years = [2012, 2013, 2014, 2015]
        durations = [5,5,5,5]
        min_events = [5,5,5,5]
        num_events = evaluate_EWRs.get_number_events(EWR_info, events, unique_water_years, durations, min_events)
        expected_num_events = [3,1,4,0]
        self.assertEqual(num_events, expected_num_events)
    def test_get_average_event_length(self):
        '''
        1. Test yearly average event length for test years with between 0 and 4 total events
        '''
        events = {2012: [[5]*5, [10]*5, [20]*8], 2013: [[50]*20],
                  2014: [[5]*5, [10]*5, [20]*8, [20]*8], 2015: []}
        unique_water_years = [2012, 2013, 2014, 2015]
        average_length = evaluate_EWRs.get_average_event_length(events, unique_water_years)
        expected_average_length = [6,20,6.5,0]
        self.assertEqual(average_length, expected_average_length)
    def test_get_total_days(self):
        '''
        1. Test total yearly event length for test years with between 0 and 4 total events
        '''
        events = {2012: [[5]*5, [10]*5, [20]*8], 2013: [[50]*20],
                  2014: [[5]*5, [10]*5, [20]*8, [20]*8], 2015: []}
        unique_water_years = [2012, 2013, 2014, 2015]
        total_days = evaluate_EWRs.get_total_days(events, unique_water_years)
        expected_total_days = [18,20,26,0]
        self.assertEqual(total_days, expected_total_days)
    def test_get_data_gap(self):
        '''
        1. Check event gaps are accurate
        '''
        data = {'409102': list(range(0,3650,10)), '425012': list(range(0,3650,10))}
        index = pd.date_range(start='1/1/2019', end='31/12/2019')
        df = pd.DataFrame(data = data, index = index)
        df.iloc[0:4] = None
        df.iloc[57] = np.nan
        unique_water_years = [2018]*181+[2019]*184
        gauge = '409102'
        missing_list = evaluate_EWRs.get_data_gap(df, unique_water_years, gauge)
        expected_missing = [5,0]
        self.assertEqual(missing_list, expected_missing)
    def test_check_flow(self):
        '''
        1. Test flow threshold passes and event requirements just met
        2. TO-TEST: flow threshold below but event requirement passed
        3. TO-TEST: event requirements failed but there is some gap track remaining
        4. TO-TEST: flow threshold failed and event requirements not met
        '''
        # Set up inputs parameters and pass to test function
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'gap_tolerance': 0, 'min_event':10}
        iteration = 50
        flow = 5
        event = [5]*9
        all_events = {2012:[[10]*10, [15]*12], 2013:[[10]*50], 
                      2014:[[10]*10, [15]*15, [10]*20], 2015:[]}
        no_event = 50
        all_no_events = {2012:[[25], [2]], 2013:[[250]],
                         2014:[[400], [2], [25]], 2015:[[450]]}
        gap_track = 0
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*365)
        total_event = 9
        event, all_events, no_event, all_no_events, gap_track, total_event = evaluate_EWRs.flow_check(EWR_info, iteration, flow, event, all_events, no_event, all_no_events, gap_track, water_years, total_event)
        # Set up expected results and test
        expected_event = [5]*10
        expected_all_events = {2012:[[10]*10, [15]*12], 2013:[[10]*50], 
                               2014:[[10]*10, [15]*15, [10]*20], 2015:[]}
        expected_no_event = 51
        expected_all_no_events = {2012:[[25], [2]], 2013:[[250]],
                                  2014:[[400], [2], [25]], 2015:[[450]]}
        expected_gap_track = 0
        expected_total_event = 10
        self.assertEqual(event,expected_event)
        for year in all_events:
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
        self.assertEqual(no_event, expected_no_event)
        self.assertDictEqual(all_no_events, expected_all_no_events)
        for year in all_no_events:
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(gap_track, expected_gap_track)
        self.assertEqual(total_event, expected_total_event)
        
    def test_lowflow_check(self):
        '''
        1. Test flow passes and event requirement just met
        2. TO-TEST: flow threshold below but event requirement passed
        3. TO-TEST: flow threshold failed and event requirements failed
        '''
        # Set up variables for all tests
        EWR_info = {'min_flow': 5, 'max_flow': 20}
        flow = 5
        water_year = 2015
        event = [5]*9
        iteration = 365+365+365+100
        all_events = {2012:[[10]*10, [15]*12], 2013:[[10]*50], 
                      2014:[[10]*10, [15]*15, [10]*20], 2015:[]}
        no_event = 0
        all_no_events = {2012:[[25], [2]], 2013:[[250]],
                         2014:[[400], [2], [25]], 2015:[[450]]}
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*365)
        event, all_events, no_event, all_no_events = evaluate_EWRs.lowflow_check(EWR_info, iteration, flow, event, all_events, no_event, all_no_events, water_years)
        # Set up expected output and test
        expected_event = [5]*10
        expected_all_events = {2012:[[10]*10, [15]*12], 2013:[[10]*50], 
                               2014:[[10]*10, [15]*15, [10]*20], 2015:[]}
        expected_no_event = 0
        expected_all_no_events = {2012:[[25], [2]], 2013:[[250]],
                                  2014:[[400], [2], [25]], 2015:[[450]]}
        self.assertEqual(event,expected_event)
        for year in all_events:
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
        self.assertEqual(no_event, expected_no_event)
        self.assertDictEqual(all_no_events, expected_all_no_events)
        for year in all_no_events:
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
    def test_ctf_check(self):
        '''
        1. flow threshold fails but event meets requirements
        2. TO-TEST: flow threshold passed
        3. TO-TEST: flow threshold failed but no event recorded
        '''
        # Set up input variables and pass to test function
        EWR_info = {'min_flow': 0, 'max_flow': 1}
        flow = 2
        event = [0]*9
        iteration = 365+365+365+100
        all_events = {2012:[[10]*10, [15]*12], 2013:[[10]*50], 
                      2014:[[10]*10, [15]*15, [10]*20], 2015:[]}
        no_event = 10
        all_no_events = {2012:[[25], [2]], 2013:[[250]],
                         2014:[[400], [2], [25]], 2015:[[450]]}
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*365)

        event, all_events, no_event, all_no_events = evaluate_EWRs.ctf_check(EWR_info, iteration, flow, event, all_events, no_event, all_no_events, water_years)
        # Set up expected outputs and test
        expected_event = []
        expected_all_events = {2012:[[10]*10, [15]*12], 2013:[[10]*50],
                               2014:[[10]*10, [15]*15, [10]*20], 2015:[[0]*9]}
        expected_no_event = 1
        expected_all_no_events = {2012:[[25], [2]], 2013:[[250]],
                                  2014:[[400], [2], [25]], 2015:[[450], [10]]}
        self.assertEqual(event,expected_event)
        for year in all_events:
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
        self.assertEqual(no_event, expected_no_event)
        for year in all_no_events:
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
    def test_level_check(self):
        '''
        1. Test level threshold fails but event requirement passed
        2. TO-TEST: test level threshold passes and event requirement met
        3. TO-TEST: level threshold fails and event requirement failed
        '''
        # Set up input variables
        EWR_info = {'min_level': 10, 'max_level':20, 'duration': 5, 'drawdown_rate':0.04}
        level = 5
        level_change = 0.04
        water_year = 2015
        event = [10]*5
        iteration = 365+365+365+100
        all_events = {2012:[[10]*10, [15]*12], 2013:[[10]*50], 
                      2014:[[10]*10, [15]*15, [10]*20], 2015:[]}
        no_event = 15
        all_no_events = {2012:[[25], [2]], 2013:[[250]],
                         2014:[[400], [2], [25]], 2015:[[450]]}
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*365)

        event, all_events, no_event, all_no_events = evaluate_EWRs.level_check(EWR_info, iteration, level, level_change, event, all_events, no_event, all_no_events, water_years)\
        # Expected results - TEST 1: #
        expected_event = []
        expected_all_events = {2012:[[10]*10, [15]*12], 2013:[[10]*50], 
                               2014:[[10]*10, [15]*15, [10]*20], 2015:[[10]*5]}
        expected_no_event = 1
        expected_all_no_events = {2012:[[25], [2]], 2013:[[250]],
                                  2014:[[400], [2], [25]], 2015:[[450], [10]]}
        self.assertEqual(event,expected_event)
        for year in all_events:
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
        self.assertEqual(no_event, expected_no_event)
        self.assertDictEqual(all_no_events, expected_all_no_events)
        for year in all_no_events:
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
                
    def test_flow_check_sim(self):
        '''
        1. flow threshold below for both sites but event requirement passed
        2. TO-TEST: Test flow threshold passes for both sites and event requirements just met
        3. TO-TEST: event requirements failed but there is some gap track remaining
        4. TO-TEST: flow threshold failed and event requirements not met
        5. TO-TEST: flow threshold above for one site and below for another, event requirements not met
        '''
        # Set up input parameters and send to test function
        iteration = 370
        EWR_info1 = {'min_flow': 10, 'max_flow': 20, 'min_event': 5, 'gap_tolerance': 0}
        EWR_info2 = {'min_flow': 10, 'max_flow': 20, 'min_event': 5, 'gap_tolerance': 0}
        flow1 = 5
        flow2 = 7
        event = [10]*5
        total_event = 5
        all_events = {2012:[[10]*10, [15]*12], 2013:[[10]*50], 
                      2014:[[10]*10, [15]*15, [10]*20], 2015:[]}
        no_event = 25
        all_no_events = {2012:[[25], [2]], 2013:[[250]],
                         2014:[[400], [2], [25]], 2015:[[450]]}
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*365)
        gap_track = 0
        event, all_events, no_event, all_no_events, gap_track, total_event = evaluate_EWRs.flow_check_sim(iteration, EWR_info1, EWR_info2, water_years, flow1, flow2, event, all_events, no_event, all_no_events, gap_track, total_event)
        # Set up expected results and test
        expected_event = []
        expected_all_events = {2012:[[10]*10, [15]*12], 2013:[[10]*50, [10]*5],
                               2014:[[10]*10, [15]*15, [10]*20], 2015:[[10]*5]}
        expected_no_event = 1
        expected_all_no_events = {2012:[[25], [2]], 2013:[[250], [20]],
                                  2014:[[400], [2], [25]], 2015:[[450]]}
        expected_gap_track = 0
        expected_total_event = 0
        self.assertEqual(gap_track, expected_gap_track)
        self.assertEqual(event,expected_event)
#         print(all_events)
#         print(expected_all_events)
        for year in all_events:
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
        self.assertEqual(no_event, expected_no_event)
        self.assertDictEqual(all_no_events, expected_all_no_events)
        for year in all_no_events:
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
                
    def test_flow_calc(self):
        '''
        1. Test functions ability to identify and save all events and event gaps for series of flows (TO-TEST: flows overlapping water year edge)
        2. TO-TEST: constrain timing window
        '''
        # Test 1
        # Set up input data
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'gap_tolerance': 0, 'min_event':10, 'duration': 10}
        flows = np.array([0]*355+[10]*10 + [0]*355+[10]*10 + [0]*355+[10]*10 + [0]*355+[10]*10+[10]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        # Set up expected output data        
        expected_all_events = {2012: [[10]*10], 2013: [[10]*10], 2014: [[10]*10], 2015: [[10]*11]}
        expected_all_no_events = {2012: [[355]], 2013: [[355]], 2014: [[355]], 2015: [[355]]}
        expected_durations = [10]*4
        expected_min_events = [10]*4
        # Send inputs to test function and test
        all_events, all_no_events, durations, min_events = evaluate_EWRs.flow_calc(EWR_info, flows, water_years, dates, masked_dates)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)
        
    def test_lowflow_calc(self):
        '''
        1. Test functions ability to identify and save all events and event gaps for series of flows
        2. Constrain timing window and test functions ability to identify and save all events and event gaps for series of flows
        '''
        # Test 1
        # set up input data 
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'min_event':1, 'duration': 300, 'duration_VD': 10}
        flows = np.array([5]*295+[0]*25+[10]*45 + [0]*355+[5000]*10 + [0]*355+[10]*10 + [5]*295+[0]*25+[10]*45+[10]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        climates = np.array(['Wet']*365 + ['Very Wet']*365 + ['Very Dry']*365 + ['Dry']*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        # Set up expected output data
        expected_all_events = {2012: [[5]*295, [10]*45], 2013: [], 2014: [[10]*10], 2015: [[5]*295, [10]*46]}
        expected_all_no_events = {2012: [[25]], 2013: [], 2014: [[720]], 2015: [[25]]}
        expected_durations = [300,300,10,300] # adding in a very dry year climate year
        expected_min_events = [1,1,1,1]
        # Send inputs to test function and test
        all_events, all_no_events, durations, min_events = evaluate_EWRs.lowflow_calc(EWR_info, flows, water_years, climates, dates, masked_dates)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)
        #------------------------------------------------
        # Test 2
        # Set up input data
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'min_event':1, 'duration': 10, 
                    'duration_VD': 5, 'start_month': 7, 'end_month': 12, 'start_day': None, 'end_day': None}
        flows = np.array([10]*5+[0]*35+[5]*5+[0]*295+[0]*25 + [0]*355+[5]*10 + [10]*10+[0]*355 + [5]*295+[0]*25+[10]*45+[10]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        climates = np.array(['Wet']*365 + ['Very Wet']*365 + ['Very Dry']*365 + ['Dry']*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = masked_dates[((masked_dates.month >= 7) & (masked_dates.month <= 12))] # Just want the dates in the date range
        # Set up expected output data
        expected_all_events = {2012: [[10]*5, [5]*5], 2013: [], 2014: [[10]*10], 2015: [[5]*184]}
        expected_all_no_events = {2012: [[35]], 2013: [[685]], 2014: [[355]], 2015: [[181]]}
        expected_durations = [10,10,5,10] # adding in a very dry year climate year
        expected_min_events = [1,1,1,1]
        # Send to test function and test
        all_events, all_no_events, durations, min_events = evaluate_EWRs.lowflow_calc(EWR_info, flows, water_years, climates, dates, masked_dates)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)

    def test_ctf_calc(self):
        '''
        1. Test functions ability to identify and save all events and event gaps for series of flows, 
           ensuring events are cut off at the end of the water year even though dates are not constrained
        2. Constrain timing window and test functions ability to identify and save all events and event gaps for series of flows
        '''
        # Test 1
        # Set up input data
        EWR_info = {'min_flow': 0, 'max_flow': 1, 'min_event':5, 'duration': 20, 'duration_VD': 10}
        flows = np.array([5]*295+[0]*25+[10]*45 + [20]*355+[5000]*5+[0]*5 + [0]*355+[10]*10 + [1]*295+[20]*25+[0]*45+[0]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        climates = np.array(['Wet']*365 + ['Very Wet']*365 + ['Very Dry']*365 + ['Dry']*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        # Set up expected output data
        expected_all_events = {2012: [[0]*25], 2013: [[0]*5], 2014: [[0]*355], 2015: [[1]*295, [0]*46]}
        expected_all_no_events = {2012: [[295]], 2013: [[405]], 2014: [], 2015: [[10], [25]]}
        expected_durations = [20,20,10,20] # adding in a very dry year climate year
        expected_min_events = [5,5,5,5]
        # Send to test function and then test
        all_events, all_no_events, durations, min_events = evaluate_EWRs.ctf_calc(EWR_info, flows, water_years, climates, dates, masked_dates)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)
        #--------------------------------------------------
        # Test 2
        # Set up input data
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'min_event':1, 'duration': 10,
                    'duration_VD': 5, 'start_month': 7, 'end_month': 12, 'start_day': None, 'end_day': None}
        flows = np.array([10]*5+[0]*35+[5]*5+[0]*295+[0]*25 + [0]*355+[5]*10 + [10]*10+[0]*355 + [5]*295+[0]*25+[10]*45+[10]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        climates = np.array(['Wet']*365 + ['Very Wet']*365 + ['Very Dry']*365 + ['Dry']*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = masked_dates[((masked_dates.month >= 7) & (masked_dates.month <= 12))] # Just want the dates in the date range
        # Set up expected output data
        expected_all_events = {2012: [[10]*5, [5]*5], 2013: [], 2014: [[10]*10], 2015: [[5]*184]}
        expected_all_no_events = {2012: [[35]], 2013: [[685]], 2014: [[355]], 2015: [[181]]}
        expected_durations = [10,10,5,10] # adding in a very dry year climate year
        expected_min_events = [1,1,1,1]
        # Send to test function and then test
        all_events, all_no_events, durations, min_events = evaluate_EWRs.lowflow_calc(EWR_info, flows, water_years, climates, dates, masked_dates)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)

    def test_ctf_calc_anytime(self):
        '''
        1. Test functions ability to identify and save all events and event gaps for series of flows, ensure events overlapping water year edges are registered
        '''
        # Set up input data
        EWR_info = {'min_flow': 0, 'max_flow': 1, 'min_event':5, 'duration': 20, 'duration_VD': 10}
        flows = np.array([5]*295+[0]*25+[10]*45 + [20]*355+[5000]*5+[0]*5 + [0]*355+[10]*10 + [1]*295+[20]*25+[0]*45+[0]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        climates = np.array(['Wet']*365 + ['Very Wet']*365 + ['Very Dry']*365 + ['Dry']*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        # Set up expected output data
        expected_all_events = {2012: [[0]*25], 2013: [], 2014: [[0]*360], 2015: [[1]*295, [0]*46]}
        expected_all_no_events = {2012: [[295]], 2013: [[405]], 2014: [], 2015: [[10], [25]]}
        expected_durations = [20,20,10,20] # adding in a very dry year climate year
        expected_min_events = [5,5,5,5]
        # Send to test function and then test
        all_events, all_no_events, durations, min_events = evaluate_EWRs.ctf_calc_anytime(EWR_info, flows, water_years, climates)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)
        
    def test_flow_calc_anytime(self):
        '''
        1. Test functions ability to identify and save all events and event gaps for series of flows, ensure events overlapping water year edges are registered
        '''
        # Set up input data
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'gap_tolerance': 0, 'min_event':10, 'duration': 10}
        flows = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*10 + [10]*5+[0]*350+[10]*10+[10]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        # Set up expected output data
        expected_all_events = {2012: [[10]*10], 2013: [], 2014: [[10]*20, [10]*15], 2015: [[10]*11]}
        expected_all_no_events = {2012: [[350]], 2013: [[360]], 2014: [[345]], 2015: [[350]]}
        expected_durations = [10]*4
        expected_min_events = [10]*4
        # Send to test function and then test
        all_events, all_no_events, durations, min_events = evaluate_EWRs.flow_calc_anytime(EWR_info, flows, water_years)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)
        
    def test_lake_calc(self):
        '''
        1. Test functions ability to identify and save all events and event gaps for series of lake levels, 
           ensuring events are cut off at the end of the water year even though dates are not constrained
        '''
        # Set up input data
        EWR_info = {'min_level': 50, 'max_level': 60, 'duration': 10, 'min_event': 10, 'drawdown_rate': 0.04}
        levels = np.array([0]*350+[50]*10+[0]*5 + [0]*355+[61]*10 + [50]*10+[0]*345+[50]*10 + [50]*5+[0]*350+[50]*10+[50]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        # Set up expected output data
        expected_all_events = {2012: [[50]*10], 2013: [], 2014: [[50]*10], 2015: [[50]*11]}
        expected_all_no_events = {2012: [[350]], 2013: [], 2014: [[725]], 2015: [[355]]}
        expected_durations = [10]*4
        expected_min_events = [10]*4
        # Send to test function and then test
        all_events, all_no_events, durations, min_events = evaluate_EWRs.lake_calc(EWR_info, levels, water_years, dates, masked_dates)
#         print(all_events)
#         print(expected_all_events)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
#         print(all_no_events)
#         print(expected_all_no_events)
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)

    def test_cumulative_calc(self):
        '''
        1. Test functions ability to identify and save all events and event gaps for series of flows, 
           ensuring events are cut off at the end of the water year even though dates are not constrained
        '''
        # Set up input data
        EWR_info = {'min_volume': 100, 'min_flow': 50, 'min_event': 2, 'duration': 2}
        flows = np.array([0]*350+[10]*10+[20]*5 + [0]*360+[100]*5 + [75]*1+[25]*1+[0]*353+[50]*10 + [50]*2+[0]*362+[49]*1+[100]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        # Set up expected output data
        expected_all_events = {2012: [], 
                               2013: [[100], [100]*2, [100]*2], 
                               2014: [[50]*2, [50]*2, [50]*2, [50]*2, [50]*2], 
                               2015: [[50]*2, [100]*1]}
        expected_all_no_events = {2012: [], 2013: [[724]], 2014: [[355]], 2015: [[362]]}
        expected_durations = [2]*4
        expected_min_events = [2]*4
        all_events, all_no_events, durations, min_events = evaluate_EWRs.cumulative_calc(EWR_info, flows, water_years, dates, masked_dates)
#         print(all_events)
#         print(expected_all_events)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
#         print(all_no_events)
#         print(expected_all_no_events)
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)
        
        
    def test_cumulative_calc_anytime(self):
        '''
        1. Test functions ability to identify and save all events and event gaps for series of flows, 
           ensuring events crossing water years are identified and registered
               - Test event crossing water years
               - Test event on final day of series
               - TO-TEST: event on first day of series
        '''
        # Set up input data
        EWR_info = {'min_volume': 100, 'min_flow': 50, 'min_event': 2, 'duration': 2}
        flows = np.array([0]*350+[10]*14+[50]*1 + [50]*1+[0]*358+[100]*6 + [75]*1+[25]*1+[0]*353+[50]*10 + [50]*2+[0]*362+[49]*1+[100]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        # Set up expected outputs
        expected_all_events = {2012: [], 
                               2013: [[50]*2, [100]*1, [100]*2, [100]*2], 
                               2014: [[100,75], [50]*2, [50]*2, [50]*2, [50]*2, [50]*2], 
                               2015: [[50]*2, [100]*1]}
        expected_all_no_events = {2012: [], 2013: [[364], [357]], 2014: [[354]], 2015: [[362]]}
        expected_durations = [2]*4
        expected_min_events = [2]*4
        # Send inputs to test function and then test
        all_events, all_no_events, durations, min_events = evaluate_EWRs.cumulative_calc_anytime(EWR_info, flows, water_years)       
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)
        
    def test_nest_calc_weirpool(self):
        '''
        1. Test functions ability to identify and save all events and event gaps for series of flows and levels, ensure events cannot overlap water years. Other tests:
            - check if event exluded when flow requirement is passed but the level requirement is not passed
            - TO-TEST: check if event exluded when flow requirement is not passed but the level requirement is passed
            - TO-TEST: check if event is excluded when flow and level requirements are passed but the drawdown rate is exceeded
        '''
        # Set up input data
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'drawdown_rate': 0.04, 'min_event': 10, 'duration': 10}
        flows = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*10 + [10]*5+[0]*351+[10]*10)
        levels = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*9+[1]*1 + [10]*5+[0]*351+[10]*10)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        # Set up expected output data
        expected_all_events = {2012: [[10]*10], 2013: [[10]*10], 2014: [[10]*10], 2015: [[10]*10]}
        expected_all_no_events = {2012: [[350]], 2013: [[360]], 2014: [], 2015: [[711]]}
        expected_durations = [10]*4
        expected_min_events = [10]*4
        # Send to test function and then test
        all_events, all_no_events, durations, min_events = evaluate_EWRs.nest_calc_weirpool(EWR_info, flows, levels, water_years, dates, masked_dates)
#         print(all_events)
#         print(expected_all_events)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
#         print(all_no_events)
#         print(expected_all_no_events)
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)
        
    def test_nest_calc_percent(self):
        '''
        1. Test functions ability to identify and save all events and event gaps for series of flows, ensure events cannot overlap water years. Other tests:
            - check if event exluded when flow requirement is passed but the drawdown rate is exceeded
        '''
        # Set up input data
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'drawdown_rate': '10%', 'min_event': 10, 'duration': 10}
        flows = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*9+[8]*1 + [10]*9+[9]*1+[0]*346+[10]*10)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        # Set up expected output data
        expected_all_events = {2012: [[10]*10], 2013: [[10]*10], 2014: [[10]*10], 2015: [[10]*9+[9]*1, [10]*10]}
        expected_all_no_events = {2012: [[350]], 2013: [[360]], 2014: [], 2015: [[355], [346]]}
        expected_durations = [10]*4
        expected_min_events = [10]*4
        # Send to test function and then test
        all_events, all_no_events, durations, min_events = evaluate_EWRs.nest_calc_percent(EWR_info, flows, water_years, dates, masked_dates)
#         print(all_events)
#         print(expected_all_events)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
#         print(all_no_events)
#         print(expected_all_no_events)
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)
        
    def test_nest_calc_percent_trigger(self):
        '''
        1. Test functions ability to identify and save all events and event gaps for series of flows, ensure events cannot overlap water years. Other tests:
            - check if event exluded when flow requirement is passed but the drawdown rate is exceeded
        '''
        # Set up input data
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'drawdown_rate': '10%', 'min_event': 10, 'duration': 10, 'trigger_day': 15, 'trigger_month': 10}
        flows = np.array([0]*106+[11]*1+[10]*9+[0]*249 + [0]*106+[9]*1+[10]*9+[0]*249 + [0]*106+[10]*9+[9]*1+[0]*249 + [0]*106+[10]*9+[8]*1+[0]*250)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        # Set up expected output data
        expected_all_events = {2012: [[11]*1+[10]*9], 2013: [[9]*1+[10]*9], 2014: [[10]*9+[9]*1], 2015: []}
        expected_all_no_events = {2012: [[106]], 2013: [[355]], 2014: [[355]], 2015: [[615]]}
        expected_durations = [10]*4
        expected_min_events = [10]*4
        # Send input data to test function and then test
        all_events, all_no_events, durations, min_events = evaluate_EWRs.nest_calc_percent_trigger(EWR_info, flows, water_years, dates)
#         print(all_events)
#         print(expected_all_events)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
#         print(all_no_events)
#         print(expected_all_no_events)
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)
        
    def test_weirpool_calc(self):
        '''
        1. Test weirpool drawdown
        2. Test weirpool raising
        
        For the above two tests: Test functions ability to identify and save all events and event gaps for series of flows and levels, ensure events cannot overlap water years. Other tests:
            - check if event exluded when flow requirement is passed but the level requirement is not passed
            - check if event is excluded when flow and level requirements are passed but the drawdown rate is exceeded
            - TO-TEST: check if event exluded when flow requirement is not passed but the level requirement is passed
        '''
        # Test 1
        # Set up input data
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'min_level': 5, 'max_level': 10, 'drawdown_rate': 0.04, 'min_event': 10, 'duration': 10}
        flows = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*10 + [10]*5+[8]*5+[0]*346+[10]*10)
        levels = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*9+[1]*1 + [11]*5+[10]*5+[0]*346+[11]*1+[10]*9)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        # Set up expected output data
        expected_all_events = {2012: [[10]*10], 2013: [[10]*10], 2014: [[10]*10], 2015: []}
        expected_all_no_events = {2012: [[350]], 2013: [[360]], 2014: [], 2015: [[721]]}
        expected_durations = [10]*4
        expected_min_events = [10]*4
        weirpool_type = 'falling'
        # Send to test function and then test
        all_events, all_no_events, durations, min_events = evaluate_EWRs.weirpool_calc(EWR_info, flows, levels, water_years, weirpool_type, dates, masked_dates)
#         print(all_events)
#         print(expected_all_events)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
#         print(all_no_events)
#         print(expected_all_no_events)
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)
        #--------------------------------------------------
        # Test 2
        # Set up input data
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'min_level': 10, 'max_level': 20, 'drawdown_rate': 0.04, 'min_event': 10, 'duration': 10}
        flows = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*10 + [10]*5+[8]*5+[0]*346+[10]*10)
        levels = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*9+[1]*1 + [11]*5+[10.96]*5+[0]*346+[10]*9+[9.96]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        # Set up expected output data
        expected_all_events = {2012: [[10]*10], 2013: [[10]*10], 2014: [[10]*10], 2015: [[10]*5+[8]*5]}
        expected_all_no_events = {2012: [[350]], 2013: [[360]], 2014: [], 2015: [[355], [356]]}
        expected_durations = [10]*4
        expected_min_event = [10]*4
        weirpool_type = 'raising'
        # Send to test function and then test
        all_events, all_no_events, durations, min_events = evaluate_EWRs.weirpool_calc(EWR_info, flows, levels, water_years, weirpool_type, dates, masked_dates)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)
        
    def test_flow_calc_anytime_sim(self):
        '''
        1. Test functions ability to identify and save all events and event gaps for series of flows
            - Check events can span multiple water years
            - Check event not registered when only one site meets flow requirement
        '''
        # Set up input data
        EWR_info1 = {'min_flow': 10, 'max_flow': 20, 'gap_tolerance': 0, 'min_event':10, 'duration': 10}
        EWR_info2 = {'min_flow': 20, 'max_flow': 30, 'gap_tolerance': 0, 'min_event':10, 'duration': 10}
        flows1 = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*10 + [10]*5+[0]*351+[10]*10)
        flows2 = np.array([0]*350+[30]*10+[0]*5 + [0]*355+[30]*10 + [30]*10+[0]*345+[10]*10 + [10]*10+[0]*346+[30]*10)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        # Set up expected output data
        expected_all_events = {2012: [[10]*10], 2013: [], 2014: [[10]*20], 2015: [[10]*10]}
        expected_all_no_events = {2012: [[350]], 2013: [[360]], 2014: [], 2015: [[711]]}
        expected_durations = [10]*4
        expected_min_events = [10]*4
        # Send to test function and then test
        all_events, all_no_events, durations, min_events = evaluate_EWRs.flow_calc_anytime_sim(EWR_info1, EWR_info2, flows1, flows2, water_years)
#         print(all_events)
#         print(expected_all_events)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
#         print(all_no_events)
#         print(expected_all_no_events)
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)

    def test_flow_calc_sim(self):
        '''
        1. Test functions ability to identify and save all events and event gaps for series of flows
            - Check events cannot span multiple water years
            - Check event not registered when only one site meets flow requirement
            - TO-TEST: constrain months of water year and repeat test
        '''
        # Set up input data
        EWR_info1 = {'min_flow': 10, 'max_flow': 20, 'gap_tolerance': 0, 'min_event':10, 'duration': 10}
        EWR_info2 = {'min_flow': 20, 'max_flow': 30, 'gap_tolerance': 0, 'min_event':10, 'duration': 10}
        flows1 = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*10 + [10]*5+[0]*351+[10]*10)
        flows2 = np.array([0]*350+[30]*10+[0]*5 + [0]*355+[30]*10 + [30]*10+[0]*345+[10]*10 + [10]*10+[0]*346+[30]*10)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        # Set up expected output data
        expected_all_events = {2012: [[10]*10], 2013: [[10]*10], 2014: [[10]*10], 2015: [[10]*10]}
        expected_all_no_events = {2012: [[350]], 2013: [[360]], 2014: [], 2015: [[711]]}
        expected_durations = [10]*4
        expected_min_events = [10]*4
        # Send to test function and then test
        all_events, all_no_events, durations, min_events = evaluate_EWRs.flow_calc_sim(EWR_info1, EWR_info2, flows1, flows2, water_years, dates, masked_dates)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertListEqual(event, expected_all_events[year][i])
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertListEqual(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)
        
    def test_lowflow_calc_sim(self):
        '''
        1. Test functions ability to identify and save all events and event gaps for series of flows
            - Test to ensure it does not matter event sequencing at each site, as long as minimum day duration is met for each year, event should be registered
        '''
        # Set up input data
        EWR_info1 = {'min_flow': 10, 'max_flow': 20, 'min_event': 1, 'duration': 10, 'duration_VD': 5}
        EWR_info2 = {'min_flow': 20, 'max_flow': 30, 'min_event': 1, 'duration': 10, 'duration_VD': 5}
        flows1 = np.array([10]*1+[0]*350+[10]*9+[0]*5 + [0]*360+[10]*5 + [10]*10+[0]*345+[10]*10 + [8]*5+[0]*351+[10]*10)
        flows2 = np.array([25]*1+[0]*350+[30]*9+[0]*5 + [0]*360+[30]*5 + [30]*10+[0]*345+[10]*10 + [18]*10+[0]*346+[30]*10)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        climates = np.array(['Wet']*365 + ['Very Dry']*365 +['Very Wet']*365 + ['Dry']*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        # Set up expected output data
        expected_all_events1 = {2012: [[10]*1, [10]*9], 2013: [[10]*5], 2014: [[10]*10, [10]*10], 2015: [[10]*10]}
        expected_all_events2 = {2012: [[25]*1, [30]*9], 2013: [[30]*5], 2014: [[30]*10], 2015: [[30]*10]}
        expected_all_no_events1 = {2012: [[350]], 2013: [[365]], 2014: [[345]], 2015: [[356]]}
        expected_all_no_events2 = {2012: [[350]], 2013: [[365]], 2014: [], 2015: [[711]]}
        expected_durations = [10,5,10,10]
        expected_min_events = [1,1,1,1]
        # Send inputs to function and then test:
        all_events1, all_events2, all_no_events1, all_no_events2, durations, min_events = evaluate_EWRs.lowflow_calc_sim(EWR_info1, EWR_info2, flows1, flows2, water_years, climates, dates, masked_dates)
#         print(all_events1)
#         print(expected_all_events1)
        for year in all_events1:
            self.assertEqual(len(all_events1[year]), len(expected_all_events1[year]))
            for i, event in enumerate(all_events1[year]):
                self.assertListEqual(event, expected_all_events1[year][i])
#         print(all_events2)
#         print(expected_all_events2)
        for year in all_events2:
            self.assertEqual(len(all_events2[year]), len(expected_all_events2[year]))
            for i, event in enumerate(all_events2[year]):
                self.assertListEqual(event, expected_all_events2[year][i])
#         print(all_no_events1)
#         print(expected_all_no_events1)
        for year in all_no_events1:
            self.assertEqual(len(all_no_events1[year]), len(expected_all_no_events1[year]))
            for i, no_event in enumerate(all_no_events1[year]):
                self.assertListEqual(no_event, expected_all_no_events1[year][i])
#         print(all_no_events2)
#         print(expected_all_no_events2)
        for year in all_no_events2:
            self.assertEqual(len(all_no_events2[year]), len(expected_all_no_events2[year]))
            for i, no_event in enumerate(all_no_events2[year]):
                self.assertListEqual(no_event, expected_all_no_events2[year][i])
        self.assertEqual(durations, expected_durations)
        self.assertEqual(min_events, expected_min_events)
        
        
    def test_ctf_calc_sim(self):
        '''
        1. Test functions ability to identify and save all events and event gaps for series of flows
        '''
        # Set up input data
        EWR_info1 = {'min_flow': 0, 'max_flow': 1, 'min_event': 10, 'duration': 10, 'duration_VD': 5}
        EWR_info2 = {'min_flow': 0, 'max_flow': 1, 'min_event': 10, 'duration': 10, 'duration_VD': 5}
        flows1 = np.array([10]*1+[0]*350+[10]*9+[1]*5 + [0]*360+[10]*5 + [10]*10+[0]*345+[10]*1+[1]*9 + [8]*5+[10]*351+[0]*10)
        flows2 = np.array([10]*1+[0]*350+[10]*9+[1]*5 + [0]*360+[10]*5 + [10]*10+[0]*345+[10]*1+[1]*9 + [8]*5+[10]*351+[0]*10)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        climates = np.array(['Wet']*365 + ['Very Dry']*365 +['Very Wet']*365 + ['Dry']*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        # Set up expected outputs
        expected_all_events1 = {2012: [[0]*350, [1]*5], 2013: [[0]*360], 2014: [[0]*345, [1]*9], 2015: [[0]*10]}
        expected_all_events2 = {2012: [[0]*350, [1]*5], 2013: [[0]*360], 2014: [[0]*345, [1]*9], 2015: [[0]*10]}
        expected_all_no_events1 = {2012: [[1], [9]], 2013: [], 2014: [[15], [1]], 2015: [[356]]}
        expected_all_no_events2 = {2012: [[1], [9]], 2013: [], 2014: [[15], [1]], 2015: [[356]]}
        expected_durations = [10,5,10,10]
        min_events = [10,10,10,10]
        # Send inputs to function and then test
        all_events1, all_events2, all_no_events1, all_no_events2, durations, min_events = evaluate_EWRs.ctf_calc_sim(EWR_info1, EWR_info2, flows1, flows2, water_years, climates, dates, masked_dates)
#         print(all_events1)
#         print(expected_all_events1)
        for year in all_events1:
            self.assertEqual(len(all_events1[year]), len(expected_all_events1[year]))
            for i, event in enumerate(all_events1[year]):
                self.assertListEqual(event, expected_all_events1[year][i])
#         print(all_events2)
#         print(expected_all_events2)
        for year in all_events2:
            self.assertEqual(len(all_events2[year]), len(expected_all_events2[year]))
            for i, event in enumerate(all_events2[year]):
                self.assertListEqual(event, expected_all_events2[year][i])
#         print(all_no_events1)
#         print(expected_all_no_events1)
        for year in all_no_events1:
            self.assertEqual(len(all_no_events1[year]), len(expected_all_no_events1[year]))
            for i, no_event in enumerate(all_no_events1[year]):
                self.assertListEqual(no_event, expected_all_no_events1[year][i])
#         print(all_no_events2)
#         print(expected_all_no_events2)
        for year in all_no_events2:
            self.assertEqual(len(all_no_events2[year]), len(expected_all_no_events2[year]))
            for i, no_event in enumerate(all_no_events2[year]):
                self.assertListEqual(no_event, expected_all_no_events2[year][i])
        self.assertEqual(durations, expected_durations)

        
Test = test_evaluate_EWRs()
suite = unittest.TestSuite()

suite.addTest(test_evaluate_EWRs('test_component_pull'))
suite.addTest(test_evaluate_EWRs('test_apply_correction'))
suite.addTest(test_evaluate_EWRs('test_get_EWRs'))
suite.addTest(test_evaluate_EWRs('test_mask_dates'))
suite.addTest(test_evaluate_EWRs('test_wateryear_daily'))
suite.addTest(test_evaluate_EWRs('test_which_wateryear'))
suite.addTest(test_evaluate_EWRs('test_get_duration'))
suite.addTest(test_evaluate_EWRs('test_construct_event_dict'))
suite.addTest(test_evaluate_EWRs('test_get_days_between'))
suite.addTest(test_evaluate_EWRs('test_get_event_years'))
suite.addTest(test_evaluate_EWRs('test_get_achievements'))
suite.addTest(test_evaluate_EWRs('test_get_number_events'))
suite.addTest(test_evaluate_EWRs('test_get_average_event_length'))
suite.addTest(test_evaluate_EWRs('test_get_total_days'))
suite.addTest(test_evaluate_EWRs('test_get_data_gap'))
suite.addTest(test_evaluate_EWRs('test_check_flow'))
suite.addTest(test_evaluate_EWRs('test_lowflow_check'))
suite.addTest(test_evaluate_EWRs('test_ctf_check'))
suite.addTest(test_evaluate_EWRs('test_level_check'))
suite.addTest(test_evaluate_EWRs('test_flow_check_sim'))
suite.addTest(test_evaluate_EWRs('test_flow_calc'))
suite.addTest(test_evaluate_EWRs('test_lowflow_calc'))
suite.addTest(test_evaluate_EWRs('test_ctf_calc'))
suite.addTest(test_evaluate_EWRs('test_ctf_calc_anytime'))
suite.addTest(test_evaluate_EWRs('test_flow_calc_anytime'))
suite.addTest(test_evaluate_EWRs('test_lake_calc'))
suite.addTest(test_evaluate_EWRs('test_cumulative_calc'))
suite.addTest(test_evaluate_EWRs('test_cumulative_calc_anytime'))
suite.addTest(test_evaluate_EWRs('test_nest_calc_weirpool'))
suite.addTest(test_evaluate_EWRs('test_nest_calc_percent'))
suite.addTest(test_evaluate_EWRs('test_nest_calc_percent_trigger'))
suite.addTest(test_evaluate_EWRs('test_weirpool_calc'))
suite.addTest(test_evaluate_EWRs('test_flow_calc_anytime_sim'))
suite.addTest(test_evaluate_EWRs('test_flow_calc_sim'))
suite.addTest(test_evaluate_EWRs('test_lowflow_calc_sim'))
suite.addTest(test_evaluate_EWRs('test_ctf_calc_sim'))


unittest.TextTestRunner().run(suite)

.../home/pedro/.pyenv/versions/3.8.13/lib/python3.8/unittest/case.py:633: UserWarning: Parsing '31/12/2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  method()
./home/pedro/.pyenv/versions/3.8.13/lib/python3.8/unittest/case.py:633: UserWarning: Parsing '31/12/2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  method()
................................
----------------------------------------------------------------------
Ran 36 tests in 1.411s

OK


<unittest.runner.TextTestResult run=36 errors=0 failures=0>